# Исследование гибридных методов решения симметричной задачи коммивояжера для оптимизации подбора параметров технологической системы
Студент группы ВМ-223: Баринов Даниил Сергеевич

## Библиотеки

In [ ]:
!pip install torch-geometric
!pip install -q git+https://github.com/snap-stanford/deepsnap.git

# !pip install torch-scatter -f https://data.pyg.org/whl/torch-1.13.1+cu116.html
# !pip install torch-sparse -f https://data.pyg.org/whl/torch-1.13.1+cu116.html

!pip install pyCombinatorial
!pip install python-tsp

!pip install pulp

In [ ]:
import torch
import torch.nn.functional as F
import torch.nn as nn
from torch.autograd import Variable


from scipy.spatial.distance import pdist, squareform
from sklearn.utils.class_weight import compute_class_weight

import logging
import networkx as nx
import matplotlib.pyplot as plt

In [ ]:
# GA
from pyCombinatorial.algorithm import (genetic_algorithm,
                                       local_search_2_opt,
                                       local_search_2h_opt,
                                       local_search_3_opt,
                                       local_search_4_opt,
                                       local_search_5_opt,
                                       local_search_2_opt_stochastic,
                                       ant_colony_optimization,
                                       adaptive_large_neighborhood_search,
                                       bellman_held_karp_exact_algorithm,
                                       branch_and_bound,
                                       biased_random_key_genetic_algorithm,
                                       cheapest_insertion,
                                       christofides_algorithm,
                                       clarke_wright_savings,
                                       concave_hull_algorithm,
                                       convex_hull_algorithm,
                                       elastic_net_tsp,
                                       extremal_optimization,
                                       farthest_insertion,
                                       greedy_randomized_adaptive_search_procedure,
                                       greedy_karp_steele_patching,
                                       guided_search,
                                       hopfield_network_tsp,
                                       iterated_search,
                                       karp_steele_patching,
                                       large_neighborhood_search,
                                       multifragment_heuristic,
                                       nearest_insertion,
                                       nearest_neighbour,
                                       random_insertion,
                                       random_tour,
                                       scatter_search,
                                       simulated_annealing_tsp,
                                       self_organizing_maps,
                                       space_filling_curve_h,
                                       space_filling_curve_m,
                                       space_filling_curve_s,
                                       stochastic_hill_climbing,
                                       sweep,
                                       tabu_search,
                                       truncated_branch_and_bound,
                                       tat_algorithm,
                                       variable_neighborhood_search
                                       )
from pyCombinatorial.utils import graphs, util

## Логирование

In [ ]:
def configure_logger(level='INFO', logfile=None):
    """
    Configures the logger with the specified log level and log file.

    Args:
        level (str): Log level ('DEBUG', 'INFO', 'WARNING', 'ERROR', 'CRITICAL').
        logfile (str): Path to the log file.

    Returns:
        None
    """
    logger = logging.getLogger("console")
    logger.setLevel(level)

    if not logger.handlers:
        if not logfile:
            console_handler = logging.StreamHandler()
            console_handler.setLevel(level)
            logger.addHandler(console_handler)
        else:
            file_handler = logging.FileHandler(logfile)
            file_handler.setLevel(level)
            logger.addHandler(file_handler)


def log(message, level='INFO'):
    """
    Logs a message with the specified log level.

    Args:
        message (str): Message to be logged.
        level (str): Log level ('DEBUG', 'INFO', 'WARNING', 'ERROR', 'CRITICAL').

    Returns:
        None
    """
    logger = logging.getLogger("console")

    match level:
        case 'DEBUG':
            logger.debug(message)
        case 'INFO':
            logger.info(message)
        case 'WARNING':
            logger.warning(message)
        case 'ERROR':
            logger.error(message)
        case 'CRITICAL':
            logger.critical(message)
        case _:
            logger.debug(message)

## Конфигурация бенчмарков

Handler для таймаутов

In [ ]:
import signal

class TimeoutException(Exception):   # Custom exception class
    pass

def handle_alarm(signal_number, frame):
    raise TimeoutException

signal.signal(signal.SIGALRM, handle_alarm)

signal_time = 300

times_info = {}
shortest_paths = {}

TSP Benchmark класс для TSPLIB и гибридных методов

In [ ]:
import numpy as np
import math
import os
import urllib.request
import zipfile
import gzip
import re
import time
import shutil
from pathlib import Path
from typing import Tuple, List, Optional, Dict, Any
import multiprocessing as mp

def solver_process_wrapper(solver_class, solver_params, dist_matrix, result_queue):
    """
    Инициализирует и запускает солвер, помещает результат в очередь.
    """
    try:
        solver = solver_class(dist_matrix)

        # Определяем, какой метод solve использовать
        solve_method_to_call = None
        if hasattr(solver, 'solve_hybrid') and callable(getattr(solver, 'solve_hybrid')):
            solve_method_to_call = solver.solve_hybrid
        elif hasattr(solver, 'solve') and callable(getattr(solver, 'solve')):
            solve_method_to_call = solver.solve
        # ... добавьте другие варианты ...
        else:
            raise RuntimeError(f"Cannot find suitable solve method for {solver_class.__name__}")

        # Убираем time_limit из параметров, т.к. он контролируется снаружи
        call_params = solver_params.copy()
        if 'time_limit' in call_params: del call_params['time_limit']
        if 'start_time_global' in call_params: del call_params['start_time_global']

        tour, length = solve_method_to_call(**call_params)
        result_queue.put(("success", tour, length)) # Отправляем результат
    except Exception as e:
        result_queue.put(("error", None, float('inf'), str(e))) # Отправляем ошибку

class TSPBenchmark:
    def __init__(self, data_dir: str = "tsp_data"):
        """
        Initialize the TSP benchmark environment.

        Args:
            data_dir: Directory to store/load the TSP datasets
        """
        self.data_dir = Path(data_dir)
        self.data_dir.mkdir(exist_ok=True)
        self.instances = {}  # Maps instance name to metadata
        self.optimal_tours = {}  # Maps instance name to optimal tour

    def download_instance(self, instance_name: str, url: str) -> bool:
        """
        Download a TSPLIB instance if not already present.
        Handles both plain files and gzipped files (.gz).

        Args:
            instance_name: Name of the instance (e.g., 'a280')
            url: Base URL to download from (will be modified for .gz files)

        Returns:
            True if successful, False otherwise
        """
        instance_dir = self.data_dir / instance_name
        instance_dir.mkdir(exist_ok=True)

        tsp_file = instance_dir / f"{instance_name}.tsp"
        opt_file = instance_dir / f"{instance_name}.opt.tour"

        # Handle gzipped files
        gz_url = url + ".gz"  # Append .gz to the URL

        # Download .tsp file if not exists
        if not tsp_file.exists():
            try:
                print(f"Downloading {instance_name}.tsp.gz...")
                # Download to a temporary file
                temp_gz_file = instance_dir / f"{instance_name}.tsp.gz"
                urllib.request.urlretrieve(gz_url, temp_gz_file)

                # Extract the gzipped file
                with gzip.open(temp_gz_file, 'rb') as f_in:
                    with open(tsp_file, 'wb') as f_out:
                        shutil.copyfileobj(f_in, f_out)

                # Remove the temporary file
                temp_gz_file.unlink()
                print(f"Successfully downloaded and extracted {instance_name}.tsp")
            except Exception as e:
                print(f"Error downloading {gz_url}: {e}")

                # Try without .gz extension as fallback
                try:
                    print(f"Trying without .gz extension...")
                    urllib.request.urlretrieve(url, tsp_file)
                    print(f"Successfully downloaded {instance_name}.tsp")
                except Exception as e2:
                    print(f"Error downloading {url}: {e2}")
                    return False

        # Try to download optimal tour if available
        opt_url = url.replace(".tsp", ".opt.tour") + ".gz"
        if not opt_file.exists():
            try:
                print(f"Downloading {instance_name}.opt.tour.gz...")
                # Download to a temporary file
                temp_gz_file = instance_dir / f"{instance_name}.opt.tour.gz"
                urllib.request.urlretrieve(opt_url, temp_gz_file)

                # Extract the gzipped file
                with gzip.open(temp_gz_file, 'rb') as f_in:
                    with open(opt_file, 'wb') as f_out:
                        shutil.copyfileobj(f_in, f_out)

                # Remove the temporary file
                temp_gz_file.unlink()
                print(f"Successfully downloaded and extracted optimal tour for {instance_name}")
            except Exception as e:
                print(f"Optimal tour not available for {instance_name}: {e}")

                # Try without .gz extension as fallback
                try:
                    print(f"Trying without .gz extension...")
                    opt_url_plain = url.replace(".tsp", ".opt.tour")
                    urllib.request.urlretrieve(opt_url_plain, opt_file)
                    print(f"Successfully downloaded optimal tour for {instance_name}")
                except Exception as e2:
                    print(f"Optimal tour not available (tried plain file): {e2}")
                    # This is not a fatal error, as some instances might not have published optimal tours

        return True

    def download_tsplib_set(self, size_range: Tuple[int, int] = (100, 1000)) -> None:
        """
        Download a set of TSPLIB instances within a size range.

        Args:
            size_range: Tuple of (min_cities, max_cities)
        """
        # TSPLIB index URL
        tsplib_index = "http://comopt.ifi.uni-heidelberg.de/software/TSPLIB95/tsp/"

        # Download a few popular instances
        instances = {
            "berlin52": "http://comopt.ifi.uni-heidelberg.de/software/TSPLIB95/tsp/berlin52.tsp",
            "eil101": "http://comopt.ifi.uni-heidelberg.de/software/TSPLIB95/tsp/eil101.tsp",
            "ch130": "http://comopt.ifi.uni-heidelberg.de/software/TSPLIB95/tsp/ch130.tsp",
            "ch150": "http://comopt.ifi.uni-heidelberg.de/software/TSPLIB95/tsp/ch150.tsp",
            # "brg180": "http://comopt.ifi.uni-heidelberg.de/software/TSPLIB95/tsp/brg180.tsp",
            "a280": "http://comopt.ifi.uni-heidelberg.de/software/TSPLIB95/tsp/a280.tsp",
            "pcb442": "http://comopt.ifi.uni-heidelberg.de/software/TSPLIB95/tsp/pcb442.tsp",
            # "pr1002": "http://comopt.ifi.uni-heidelberg.de/software/TSPLIB95/tsp/pr1002.tsp"
        }

        for name, url in instances.items():
            self.download_instance(name, url)

    # The rest of the class remains unchanged
    def load_instance(self, instance_name: str) -> Dict[str, Any]:
        """
        Load a TSP instance and parse its metadata.

        Args:
            instance_name: Name of the instance to load

        Returns:
            Dictionary with instance metadata and distance matrix
        """
        tsp_file = self.data_dir / instance_name / f"{instance_name}.tsp"

        if not tsp_file.exists():
            raise FileNotFoundError(f"Instance file {tsp_file} not found")

        metadata = {
            "name": instance_name,
            "coords": [],
            "dimension": 0,
            "edge_weight_type": "",
            "comment": ""
        }

        # Parse the .tsp file
        reading_coords = False
        with open(tsp_file, 'r') as f:
            for line in f:
                line = line.strip()

                if line == "EOF":
                    break

                if reading_coords:
                    if line:
                        parts = line.split()
                        # Skip the index (first part) and parse the coordinates
                        coords = [float(p) for p in parts[1:3]]
                        metadata["coords"].append(coords)
                    continue

                if ":" in line:
                    key, value = [p.strip() for p in line.split(":", 1)]
                    if key == "DIMENSION":
                        metadata["dimension"] = int(value)
                    elif key == "EDGE_WEIGHT_TYPE":
                        metadata["edge_weight_type"] = value
                    elif key == "COMMENT":
                        metadata["comment"] = value

                if line == "NODE_COORD_SECTION":
                    reading_coords = True

        # Convert coordinates to a NumPy array
        metadata["coords"] = np.array(metadata["coords"])

        # Calculate distance matrix
        n = metadata["dimension"]
        distances = np.zeros((n, n))
        if metadata["edge_weight_type"] == "EUC_2D":
            # Euclidean distance (rounded to nearest integer as per TSPLIB standard)
            for i in range(n):
                for j in range(n):
                    if i != j:
                        dx = metadata["coords"][i][0] - metadata["coords"][j][0]
                        dy = metadata["coords"][i][1] - metadata["coords"][j][1]
                        distances[i][j] = round(math.sqrt(dx*dx + dy*dy))
        elif metadata["edge_weight_type"] == "GEO":
            # Geographical distance (latitude/longitude)
            # Implement GEO distance calculation if needed
            pass
        else:
            # Default to Euclidean distance (not rounded)
            for i in range(n):
                for j in range(n):
                    if i != j:
                        distances[i][j] = np.linalg.norm(metadata["coords"][i] - metadata["coords"][j])

        metadata["distances"] = distances

        # Try to load optimal tour if available
        opt_file = self.data_dir / instance_name / f"{instance_name}.opt.tour"
        if opt_file.exists():
            optimal_tour = self.load_optimal_tour(opt_file)
            metadata["optimal_tour"] = optimal_tour
            metadata["optimal_length"] = self.calculate_tour_length(
                optimal_tour, distances
            )
            print(f"Loaded optimal tour with length {metadata['optimal_length']}")
        else:
            print(f"No optimal tour available for {instance_name}")

        self.instances[instance_name] = metadata
        return metadata

    def load_optimal_tour(self, tour_file: Path) -> List[int]:
        """
        Load an optimal tour from a .tour file.

        Args:
            tour_file: Path to the .tour file

        Returns:
            List of city indices forming the optimal tour
        """
        tour = []
        reading_tour = False

        with open(tour_file, 'r') as f:
            for line in f:
                line = line.strip()

                if line == "EOF":
                    break

                if reading_tour:
                    if line and line.isdigit():
                        # TSPLIB tour files use 1-based indexing, convert to 0-based
                        tour.append(int(line) - 1)
                    continue

                if line == "TOUR_SECTION":
                    reading_tour = True

        return tour

    def calculate_tour_length(self, tour: List[int], distances: np.ndarray) -> float:
        """
        Calculate the length of a tour.

        Args:
            tour: List of city indices
            distances: Distance matrix

        Returns:
            Total tour length
        """
        length = 0
        for i in range(len(tour)):
            length += distances[tour[i]][tour[(i + 1) % len(tour)]]
        return length

    def benchmark_solver(self,
                         instance_name: str,
                         solver_class,
                         solver_params: Dict[str, Any] = None,
                         runs: int = 5,
                         time_limit: Optional[float] = None) -> Dict[str, Any]:
        """
        Benchmark a TSP solver on a specific instance.

        Args:
            instance_name: Name of the instance to benchmark on
            solver_class: Class of the solver to benchmark
            solver_params: Parameters to pass to the solver
            runs: Number of runs to perform
            time_limit: Maximum time in seconds allowed for each run.

        Returns:
            Dictionary with benchmark results
        """
        if instance_name not in self.instances:
            self.load_instance(instance_name)
        instance = self.instances[instance_name]
        dist_matrix = instance["distances"]
        if solver_params is None: solver_params = {}

        results = {
            "instance": instance_name, "dimension": instance["dimension"],
            "runs": [], "timeouts": 0,
            "best_length": float('inf'), "worst_length": float('-inf'),
            "mean_length": 0, "mean_time": 0
        }
        total_length = 0.0; total_time = 0.0
        completed_runs = 0

        for run in range(runs):
            print(f"  Run {run+1}/{runs} (limit: {time_limit if time_limit else 'None'}s)...", end=" ")

            timed_out = False
            tour = None; length = float('inf')
            solve_time = float(time_limit) if time_limit is not None else float('inf')
            run_status = "unknown"

            # Создаем очередь для получения результата
            result_queue = mp.Queue()
            # Создаем и запускаем процесс
            process = mp.Process(target=solver_process_wrapper, args=(
                solver_class, solver_params, dist_matrix, result_queue
            ))

            start_time = time.time()
            process.start()
            # Ждем завершения процесса ИЛИ таймаута
            process.join(timeout=time_limit) # Ждем не более time_limit секунд

            actual_run_time = time.time() - start_time

            # Проверяем, завершился ли процесс
            if process.is_alive():
                # Процесс все еще работает -> таймаут
                print("TIMEOUT", end=" ")
                timed_out = True
                process.terminate() # Принудительно завершаем
                process.join() # Ждем завершения после terminate
                solve_time = time_limit if time_limit is not None else actual_run_time
                length = float('inf')
                tour = None
                run_status = "timeout"
                results["timeouts"] += 1
            else:
                # Процесс завершился сам
                solve_time = actual_run_time
                try:
                    # Пытаемся получить результат из очереди
                    q_result = result_queue.get_nowait()
                    run_status, tour, length = q_result[0], q_result[1], q_result[2]
                    if run_status == "error":
                         print(f"ERROR ({q_result[3]})", end=" ")
                         length = float('inf'); tour = None
                    else: # Успех
                         print(f"OK (t={solve_time:.2f}s, len={length:.2f})")

                except mp.queues.Empty:
                    # Очередь пуста, хотя процесс завершился (странно, но возможно)
                    print("ERROR (Process finished but no result in queue)", end=" ")
                    length = float('inf'); tour = None
                    run_status = "error_no_result"
                except Exception as e:
                    print(f"ERROR (getting result from queue: {e})", end=" ")
                    length = float('inf'); tour = None
                    run_status = "error_queue_get"

            print() # Перевод строки после вывода статуса

            run_result = {"tour": tour, "length": length, "time": solve_time, "timed_out": timed_out, "status": run_status}
            results["runs"].append(run_result)

            # Статистика по успешным (не таймаут и не ошибка)
            if not timed_out and length != float('inf') and run_status == "success":
                completed_runs += 1
                total_length += length
                total_time += solve_time
                if length < results["best_length"]:
                    results["best_length"] = length; results["best_tour"] = tour
                results["worst_length"] = max(results.get("worst_length", float('-inf')), length)

        # Расчет средних
        if completed_runs > 0:
            results["mean_length"] = total_length / completed_runs
            results["mean_time"] = total_time / completed_runs
        else:
            results["mean_length"] = float('inf')
            results["mean_time"] = float(time_limit) if time_limit is not None else float('inf')

        print(f"  Solver Results: Timeouts={results['timeouts']}/{runs}")

        # Calculate accuracy if optimal tour is available
        if "optimal_length" in instance:
            optimal_length = instance["optimal_length"]
            results["optimal_length"] = optimal_length
            results["best_gap"] = (results["best_length"] - optimal_length) / optimal_length * 100
            results["mean_gap"] = (results["mean_length"] - optimal_length) / optimal_length * 100

            print(f"Optimal tour length: {optimal_length}")
            print(f"Best tour length: {results['best_length']} (gap: {results['best_gap']:.2f}%)")
            print(f"Mean tour length: {results['mean_length']} (gap: {results['mean_gap']:.2f}%)")
        else:
            print(f"Best tour length: {results['best_length']}")
            print(f"Mean tour length: {results['mean_length']}")

        print(f"Mean solution time: {results['mean_time']:.2f} seconds")

        return results

    def benchmark_all(self,
                      solvers: List[Tuple[str, Any, Dict[str, Any]]],
                      instances: List[str] = None,
                      runs: int = 3,
                      instance_time_limits: Optional[Dict[str, int]] = None) -> Dict[Tuple[str, str], Dict[str, Any]]:
        """
        Benchmark multiple solvers on multiple instances.

        Args:
            solvers: List of (solver_name, solver_class, solver_params) tuples
            instances: List of instance names to benchmark on (if None, use all loaded instances)
            runs: Number of runs per solver per instance

        Returns:
            Dictionary mapping (instance_name, solver_name) to benchmark results
        """
        if instances is None:
            # Use all instances that have been loaded
            instances = list(self.instances.keys())

            # If no instances are loaded, download and load some standard ones
            if not instances:
                self.download_tsplib_set()
                instances = ["berlin52", "a280", "pcb442"]
                for instance in instances:
                    self.load_instance(instance)

        if instance_time_limits is None:
            instance_time_limits = {}

        results = {}

        for instance_name in instances:
            # ---> ДОБАВЛЕНО: Получение лимита для инстанса <---
            # Используем float для time_limit, т.к. join принимает float
            time_limit_for_instance = instance_time_limits.get(instance_name)
            time_limit_for_instance_float = float(time_limit_for_instance) if time_limit_for_instance is not None else None
            limit_str = f"(Limit: {time_limit_for_instance}s)" if time_limit_for_instance else "(No Limit)"

            print(f"\n{'='*50}")
            print(f"Benchmarking on {instance_name} {limit_str}") # <--- Добавлен вывод лимита
            print(f"{'='*50}")

            if instance_name not in self.instances:
                 try: self.load_instance(instance_name)
                 except FileNotFoundError: print(f"Skipping {instance_name}, file not found."); continue

            for solver_name, solver_class, solver_params in solvers:
                print(f"\n{'-'*40}")
                print(f"Using solver: {solver_name}")
                print(f"{'-'*40}")

                key = (instance_name, solver_name)
                # ---> ИЗМЕНЕНИЕ 4: Передача time_limit_for_instance_float <---
                results[key] = self.benchmark_solver(
                    instance_name=instance_name,
                    solver_class=solver_class,
                    solver_params=solver_params,
                    runs=runs,
                    time_limit=time_limit_for_instance_float # <--- Передаем float лимит
                )
                results[key]["solver"] = solver_name
                try: results[key]["params"] = str(solver_params)
                except: results[key]["params"] = "Error converting params"

        # Печать итогов
        self.print_benchmark_summary(results)
        return results

    def print_benchmark_summary(self, results: Dict[Tuple[str, str], Dict[str, Any]]) -> None:
        """
        Print a summary of benchmark results.

        Args:
            results: Dictionary mapping (instance_name, solver_name) to benchmark results
        """
        print("\n\n")
        print("="*80)
        print("BENCHMARK SUMMARY")
        print("="*80)

        # Group by instance
        by_instance = {}
        for (instance, solver), result in results.items():
            if instance not in by_instance:
                by_instance[instance] = []
            by_instance[instance].append((solver, result))

        # Print results for each instance
        for instance, solver_results in by_instance.items():
            print(f"\nInstance: {instance} ({self.instances[instance]['dimension']} cities)")
            print("-" * 80)
            print(f"{'Solver':<20} {'Best Length':<15} {'Mean Length':<15} {'Mean Time (s)':<15} {'Gap (%)':<10}")
            print("-" * 80)

            optimal_length = None
            if "optimal_length" in self.instances[instance]:
                optimal_length = self.instances[instance]["optimal_length"]

            for solver, result in sorted(solver_results, key=lambda x: x[1]["best_length"]):
                if optimal_length is not None:
                    gap = (result["best_length"] - optimal_length) / optimal_length * 100
                    gap_str = f"{gap:.2f}%"
                else:
                    gap_str = "N/A"

                print(f"{solver:<20} {result['best_length']:<15.2f} {result['mean_length']:<15.2f} {result['mean_time']:<15.2f} {gap_str:<10}")

            if optimal_length is not None:
                print(f"\nOptimal tour length: {optimal_length}")

    def visualize_tour(self, instance_name: str, tour: List[int], title: str = None) -> None:
        """
        Visualize a tour for a given instance.

        Args:
            instance_name: Name of the instance
            tour: List of city indices representing the tour
            title: Title for the plot
        """
        try:
            import matplotlib.pyplot as plt

            if instance_name not in self.instances:
                self.load_instance(instance_name)

            instance = self.instances[instance_name]
            coords = instance["coords"]

            # Create a plot
            plt.figure(figsize=(10, 8))

            # Plot cities
            plt.scatter(coords[:, 0], coords[:, 1], c='blue', s=20)

            # Plot tour
            for i in range(len(tour)):
                city1 = tour[i]
                city2 = tour[(i + 1) % len(tour)]
                plt.plot([coords[city1, 0], coords[city2, 0]],
                         [coords[city1, 1], coords[city2, 1]],
                         'r-', alpha=0.7)

            if title:
                plt.title(title)
            else:
                plt.title(f"Tour for {instance_name}")

            plt.tight_layout()

            # Save the plot to a file
            output_dir = self.data_dir / "plots"
            output_dir.mkdir(exist_ok=True)
            plt.savefig(output_dir / f"{instance_name}_{int(time.time())}.png")

            plt.show()

        except ImportError:
            print("Matplotlib is required for visualization.")
            print("Install it with: pip install matplotlib")

## Визуализация

In [ ]:
def plot_tsp(p, x_coord, W, W_val, W_target, title="default"):
    """
    Helper function to plot TSP tours.

    Args:
        p: Matplotlib figure/subplot axis (e.g., plt.gca()).
        x_coord: Coordinates of nodes (num_nodes, 2).
        W: Edge adjacency matrix (ignored if plotting a tour path).
        W_val: Edge values (distance) matrix (used by nx.Graph).
        W_target: One-hot matrix with 1s on edges to plot (e.g., tour edges).
        title: Title of figure/subplot.

    Returns:
        p: Updated figure/subplot axis.
    """

    def _edges_to_node_pairs(W_target_matrix):
        """Helper function to convert edge matrix into pairs of adjacent nodes."""
        pairs = []
        rows, cols = np.where(W_target_matrix == 1)
        # Avoid duplicates for undirected graphs, take only (i, j) where i < j
        for r, c in zip(rows, cols):
            if r < c:
                pairs.append((r, c))
        return pairs

    # Используем nx.Graph() вместо DiGraph, т.к. TSP тур неориентированный
    G = nx.Graph(W_val) # Передаем матрицу расстояний для информации о графе
    pos = dict(zip(range(len(x_coord)), x_coord.tolist())) # Позиции узлов

    # Получаем пары ребер из матрицы W_target
    target_pairs = _edges_to_node_pairs(W_target)

    colors = ['g'] + ['b'] * (len(x_coord) - 1) # Зеленый для 0, синий для остальных
    nx.draw_networkx_nodes(G, pos, node_color=colors, node_size=50, ax=p)

    # Рисуем только ребра тура
    nx.draw_networkx_edges(G, pos, edgelist=target_pairs, alpha=1, width=1.5, edge_color='r', ax=p)

    # Добавляем подписи узлов
    labels = {i: str(i) for i in range(len(x_coord))}
    nx.draw_networkx_labels(G, pos, labels=labels, font_size=8, ax=p)

    p.set_title(title)
    p.tick_params(left=True, bottom=True, labelleft=True, labelbottom=True) # Показываем оси
    p.set_xlabel("X Coordinate")
    p.set_ylabel("Y Coordinate")
    p.grid(True, linestyle='--', alpha=0.6)
    p.axis('equal') # Равные масштабы

    return p


# --- Основная функция plot_solution ---
def plot_solution(nodes_coord: np.ndarray,
                  tour: List[int],
                  title: str,
                  tour_length: float,
                  ax: Optional[plt.Axes] = None):
    """
    Визуализирует один маршрут TSP на заданной оси matplotlib.

    Args:
        nodes_coord: Координаты узлов (N, 2).
        tour: Список индексов узлов в порядке обхода (0-based).
        tour_length: Длина маршрута.
        title: Заголовок для графика.
        ax: Ось matplotlib для рисования. Если None, создается новая фигура/ось.
    """
    n = nodes_coord.shape[0]
    if not tour:
        print(f"Warning: Cannot plot empty tour for '{title}'.")
        if ax: ax.set_title(f"{title}\n(Empty Tour)")
        return

    if len(tour) != n:
         print(f"Warning: Tour length ({len(tour)}) != num_nodes ({n}) for '{title}'. Plotting partial.")

    if ax is None:
        # Если ось не передана, создаем новую фигуру и ось
        fig, ax = plt.subplots(figsize=(8, 8))

    # Создаем матрицу W_target для ребер тура
    W_tour = np.zeros((n, n))
    for i in range(len(tour)):
        u = tour[i]
        v = tour[(i + 1) % len(tour)]
        if 0 <= u < n and 0 <= v < n:
            W_tour[u, v] = 1
            W_tour[v, u] = 1
        else:
            print(f"Error: Invalid node index in tour for plotting. u={u}, v={v}")
            ax.set_title(f"{title}\n(Invalid Tour)")
            return

    # Создаем фиктивные матрицы для plot_tsp
    dist_matrix_dummy = squareform(pdist(nodes_coord))
    W_dummy = np.ones((n,n))
    np.fill_diagonal(W_dummy, 0)

    # Используем plot_tsp для рисования на переданной оси 'ax'
    plot_tsp(ax, nodes_coord, W_dummy, dist_matrix_dummy, W_tour,
             title=f"{title}\nLength: {tour_length:.4f}")

    # Выделяем начальную точку
    start_node_idx = tour[0]
    # Используем plot вместо scatter на оси ax для лучшего контроля
    ax.plot(nodes_coord[start_node_idx, 0], nodes_coord[start_node_idx, 1],
            marker='*', color='yellow', markersize=15, markeredgecolor='black', zorder=4, linestyle='None')

# ==============================================================================
# 2. Новая функция `plot_comparison` (для двух графиков)
# ==============================================================================

def plot_comparison(nodes_coord: np.ndarray,
                    found_tour: List[int],
                    found_length: float,
                    optimal_tour: List[int],
                    optimal_length: float,
                    instance_name: str,
                    method_name: str):
    """
    Создает фигуру с двумя графиками: оптимальный тур слева, найденный тур справа.

    Args:
        nodes_coord: Координаты узлов (N, 2).
        found_tour: Найденный маршрут.
        found_length: Длина найденного маршрута.
        optimal_tour: Оптимальный маршрут.
        optimal_length: Длина оптимального маршрута.
        instance_name: Имя экземпляра TSP.
        method_name: Имя метода, который нашел `found_tour`.
    """
    if not optimal_tour:
        print("Cannot plot comparison without optimal tour.")
        # Рисуем только найденный тур
        plot_solution(nodes_coord, found_tour, f"Found ({method_name}): {instance_name}", found_length)
        return

    if not found_tour:
        print("Found tour is empty, cannot plot comparison.")
         # Рисуем только оптимальный тур
        plot_solution(nodes_coord, optimal_tour, f"Optimal: {instance_name}", optimal_length)
        return

    # Создаем фигуру с двумя колонками (1 ряд, 2 колонки)
    fig, axes = plt.subplots(1, 2, figsize=(16, 8)) # Увеличиваем размер фигуры

    # --- График 1: Оптимальный тур ---
    plot_solution(nodes_coord=nodes_coord,
                  tour=optimal_tour,
                  tour_length=optimal_length,
                  title=f"Optimal Solution: {instance_name}",
                  ax=axes[0]) # Рисуем на левой оси

    # --- График 2: Найденный тур ---
    plot_solution(nodes_coord=nodes_coord,
                  tour=found_tour,
                  tour_length=found_length,
                  title=f"Found ({method_name}): {instance_name}",
                  ax=axes[1]) # Рисуем на правой оси

    # Общий заголовок для всей фигуры
    gap = float('inf')
    if optimal_length > 0 and found_length != float('inf'):
        gap = (found_length - optimal_length) / optimal_length * 100
        fig.suptitle(f"TSP Solution Comparison: {instance_name}\nMethod: {method_name} | Gap: {gap:.2f}%", fontsize=14)
    else:
        fig.suptitle(f"TSP Solution Comparison: {instance_name}\nMethod: {method_name}", fontsize=14)


    plt.tight_layout(rect=[0, 0.03, 1, 0.95]) # Adjust layout to prevent title overlap
    plt.show()

## Точные методы

### Библиотечный метод: метод ветвей и границ (python-tsp)

In [ ]:
from python_tsp.exact import solve_tsp_branch_and_bound
print("Метод ветвей и границ (библиотека)")
signal.alarm(signal_time)
start = time.time()
try:
  path, min_length = solve_tsp_branch_and_bound(distance_matrix)
except TimeoutException:
  min_length = np.inf
  path = []
  print('Время истекло.')
else:
  # Reset the alarm
  signal.alarm(0)
end = time.time() - start

times_info["branch_and_bound_lib"] = end
shortest_paths["branch_and_bound_lib"] = f"Минимальная длина маршрута: {min_length}, Путь: {path}"
print("Минимальная стоимость пути:", min_length)
print("Лучший путь:", path)

### Библиотечный метод: метод полного перебора (python-tsp)

In [ ]:
from python_tsp.exact import solve_tsp_brute_force
print("Метод полного перебора (библиотека)")
signal.alarm(signal_time)
start = time.time()
try:
  path, distance = solve_tsp_brute_force(distance_matrix)
except TimeoutException:
  distance = np.inf
  path = []
  print('Время истекло.')
else:
  # Reset the alarm
  signal.alarm(0)
end = time.time() - start
times_info["brute_force_lib"] = end
print("Минимальная стоимость пути:", distance)
print("Лучший путь:", path)

### Библиотечный метод: метод динамического программирования (python-tsp)

In [ ]:
from python_tsp.exact import solve_tsp_dynamic_programming
print("Метод динамического программирования (библиотека)")
signal.alarm(signal_time)
start = time.time()
try:
  path, min_length = solve_tsp_dynamic_programming(distance_matrix)
except TimeoutException:
  min_length = np.inf
  path = []
  print('Время истекло.')
else:
  # Reset the alarm
  signal.alarm(0)
end = time.time() - start
times_info["dynamic_programming_lib"] = end
shortest_paths["dynamic_programming_lib"] = f"Минимальная длина маршрута: {min_length}, Путь: {path}"
print("Минимальная стоимость пути:", min_length)
print("Лучший путь:", path)

In [ ]:
# Held-Karp algorithm: O(n^2 * 2n) Not feasible to find exact solution for n >= 30 (takes several hours, which is a problem when running on colab)


## Эвристические методы

In [ ]:
from python_tsp.heuristics import solve_tsp_local_search
print("Метод локального поиска (библиотека)")
signal.alarm(signal_time)
start = time.time()
try:
  path, min_length = solve_tsp_local_search(distance_matrix)
except TimeoutException:
  min_length = np.inf
  path = []
  print('Время истекло.')
else:
  # Reset the alarm
  signal.alarm(0)
end = time.time() - start
times_info["local_search_lib"] = end
shortest_paths["local_search_lib"] = f"Минимальная длина маршрута: {min_length}, Путь: {path}"
print("Минимальная стоимость пути:", min_length)
print("Лучший путь:", path)

print(f"\nSolved instance: {instance_to_solve.name}")
print(f"  Found Length: {min_length:.4f}")
print(f"  Optimal Length: {optimal_len:.4f}")
if optimal_len > 0: # Избегаем деления на ноль
    gap = (min_length - optimal_len) / optimal_len * 100
    print(f"  Gap: {gap:.2f}%")

# Визуализация (если нужно)
if path and optimal_path:
    plot_comparison(
        nodes_coord=coordinates,
        found_tour=path,
        found_length=min_length,
        optimal_tour=optimal_path,
        optimal_length=optimal_len,
        instance_name=instance_to_solve.name,
        method_name="Local Search (python-tsp)"
    )
else:
    print("Не удалось получить один из туров для сравнения.")

In [ ]:
from python_tsp.heuristics import solve_tsp_lin_kernighan
print("Метод Лина-Кернигана (библиотека)")
signal.alarm(signal_time)
start = time.time()
try:
  path, min_length = solve_tsp_lin_kernighan(distance_matrix)
except TimeoutException:
  min_length = np.inf
  path = []
  print('Время истекло.')
else:
  # Reset the alarm
  signal.alarm(0)
end = time.time() - start
times_info["lin_kernighan_lib"] = end
shortest_paths["lin_kernighan_lib"] = f"Минимальная длина маршрута: {min_length}, Путь: {path}"
print("Минимальная стоимость пути:", min_length)
print("Лучший путь:", path)

In [ ]:
from python_tsp.heuristics import solve_tsp_simulated_annealing
print("Метод имитации отжига (библиотека)")
signal.alarm(signal_time)
start = time.time()
try:
  path, min_length = solve_tsp_simulated_annealing(distance_matrix)
except TimeoutException:
  min_length = np.inf
  path = []
  print('Время истекло.')
else:
  # Reset the alarm
  signal.alarm(0)
end = time.time() - start
times_info["simulated_annealing_lib"] = end
shortest_paths["simulated_annealing_lib"] = f"Минимальная длина маршрута: {min_length}, Путь: {path}"
print("Минимальная стоимость пути:", min_length)
print("Лучший путь:", path)

## Библиотека PyCombinatorial

### Seed

In [ ]:
# Initial Custom Seed
# seed = [[ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10,
#          11, 12, 13, 14, 15, 16, 17, 18, 19, 20,
#          21, 22, 23, 24, 25, 26, 27, 28, 29, 30,
#          31, 32, 33, 34, 35, 36, 37, 38, 39, 40,
#          41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
#          51, 52,  1] , 22205.617692710774]

# Or use the Random Seed generator
seed = util.seed_function(distance_matrix)
np.random.seed(0)

### Алгоритмы

In [ ]:
# GA - Parameters
parameters = {
            'population_size': 15,
            'elite': 1,
            'mutation_rate': 0.1,
            'mutation_search': 8,
            'generations': 100,  # Change to -> 1000
            'verbose': True
             }

# GA - Algorithm
start = time.time()
route, distance = genetic_algorithm(distance_matrix, **parameters)
end = time.time() - start

# Plot Locations and Tour
graphs.plot_tour(coordinates, city_tour = route, view = 'notebook', size = 10)
route = [x - 1 for x in route]
times_info["genetic_algorithm"] = end
shortest_paths["genetic_algorithm"] = f"Минимальная длина маршрута: {distance}, Путь: {route}"
print('Total Distance: ', round(distance, 2))
print("Route: ", route)

In [ ]:
# 2-opt - Parameters
parameters = {
            'recursive_seeding': -1, # Total Number of Iterations. If This Value is Negative Then the Algorithm Only Stops When Convergence is Reached
            'verbose': True
             }

# 2-opt - Algorithm
start = time.time()
route, distance = local_search_2_opt(distance_matrix, seed, **parameters)
end = time.time() - start

# Plot Locations and Tour
graphs.plot_tour(coordinates, city_tour = route, view = 'notebook', size = 10)
print('Total Distance: ', round(distance, 2))

route = [x - 1 for x in route]
times_info["local_search_2_opt"] = end
shortest_paths["local_search_2_opt"] = f"Минимальная длина маршрута: {distance}, Путь: {route}"

In [ ]:
# 2.5-opt - Algorithm
start = time.time()
route, distance = local_search_2h_opt(distance_matrix, seed, **parameters)
end = time.time() - start

# Plot Locations and Tour
graphs.plot_tour(coordinates, city_tour = route, view = 'notebook', size = 10)
route = [x - 1 for x in route]
times_info["local_search_2h_opt"] = end
shortest_paths["local_search_2h_opt"] = f"Минимальная длина маршрута: {distance}, Путь: {route}"
print('Total Distance: ', round(distance, 2))

In [ ]:
# 3-opt - Algorithm
start = time.time()
route, distance = local_search_3_opt(distance_matrix, seed, **parameters)
end = time.time() - start

# Plot Locations and Tour
graphs.plot_tour(coordinates, city_tour = route, view = 'notebook', size = 10)
route = [x - 1 for x in route]
times_info["local_search_3_opt"] = end
shortest_paths["local_search_3_opt"] = f"Минимальная длина маршрута: {distance}, Путь: {route}"
print('Total Distance: ', round(distance, 2))

In [ ]:
# # 4-opt - Algorithm
# start = time.time()
# route, distance = local_search_4_opt(distance_matrix, seed, **parameters)
# end = time.time() - start

# # Plot Locations and Tour
# graphs.plot_tour(coordinates, city_tour = route, view = 'notebook', size = 10)
# route = [x - 1 for x in route]
# times_info["local_search_4_opt"] = end
# shortest_paths["local_search_4_opt"] = f"Минимальная длина маршрута: {distance}, Путь: {route}"
# print('Total Distance: ', round(distance, 2))

In [ ]:
# # 5-opt - Algorithm
# start = time.time()
# route, distance = local_search_5_opt(distance_matrix, seed, **parameters)
# end = time.time() - start

# # Plot Locations and Tour
# graphs.plot_tour(coordinates, city_tour = route, view = 'notebook', size = 10)
# route = [x - 1 for x in route]
# times_info["local_search_5_opt"] = end
# shortest_paths["local_search_5_opt"] = f"Минимальная длина маршрута: {distance}, Путь: {route}"
# print('Total Distance: ', round(distance, 2))

In [ ]:
# 2-opt-stohastic - Algorithm
start = time.time()
route, distance = local_search_2_opt_stochastic(distance_matrix, seed, **parameters)
end = time.time() - start

# Plot Locations and Tour
graphs.plot_tour(coordinates, city_tour = route, view = 'notebook', size = 10)
route = [x - 1 for x in route]
times_info["local_search_2_opt_stochastic"] = end
shortest_paths["local_search_2_opt_stochastic"] = f"Минимальная длина маршрута: {distance}, Путь: {route}"
print('Total Distance: ', round(distance, 2))

In [ ]:
# ACO - Parameters
parameters = {
              'ants': 15,
              'iterations': 100,
              'alpha':1,
              'beta':2,
              'decay':0.05,
              'local_search': True,
              'verbose': True
             }

# ACO - Algorithm
start = time.time()
route, distance = ant_colony_optimization(distance_matrix, **parameters)
end = time.time() - start

# Plot Locations and Tour
graphs.plot_tour(coordinates, city_tour = route, view = 'notebook', size = 10)
route = [x - 1 for x in route]
times_info["ant_colony_optimization"] = end
shortest_paths["ant_colony_optimization"] = f"Минимальная длина маршрута: {distance}, Путь: {route}"
print('Total Distance: ', round(distance, 2))

In [ ]:
# ALNS - Parameters
parameters = {
              'iterations': 100,
              'removal_fraction':0.2,
              'rho':0.1,
              'local_search': True,
              'verbose': True
             }

# ALNS - Algorithm
start = time.time()
route, distance = adaptive_large_neighborhood_search(distance_matrix, **parameters)
end = time.time() - start

# Plot Locations and Tour
graphs.plot_tour(coordinates, city_tour = route, view = 'notebook', size = 10)
route = [x - 1 for x in route]
times_info["adaptive_large_neighborhood_search"] = end
shortest_paths["adaptive_large_neighborhood_search"] = f"Минимальная длина маршрута: {distance}, Путь: {route}"
print('Total Distance: ', round(distance, 2))

In [ ]:
# BHK - Algorithm (NOT ENOUGH RAM FOR 25 VERTICES ON CPU)
parameters = {
            'verbose': True
             }

start = time.time()
route, distance = bellman_held_karp_exact_algorithm(distance_matrix, **parameters)
end = time.time() - start

# Plot Locations and Tour
graphs.plot_tour(distance_matrix , city_tour = route, view = 'notebook', size = 10)
route = [x - 1 for x in route]
times_info["bellman_held_karp_exact_algorithm"] = end
shortest_paths["bellman_held_karp_exact_algorithm"] = f"Минимальная длина маршрута: {distance}, Путь: {route}"
print('Total Distance: ', round(distance, 2))

In [ ]:
# B&B - Algorithm
if first_test_instance.num_nodes <= 10:
  start = time.time()
  route, distance = branch_and_bound(distance_matrix)
  end = time.time() - start

  # Plot Locations and Tour
  graphs.plot_tour(distance_matrix , city_tour = route, view = 'notebook', size = 10)
  route = [x - 1 for x in route]
  times_info["branch_and_bound"] = end
  shortest_paths["branch_and_bound"] = f"Минимальная длина маршрута: {distance}, Путь: {route}"
  print('Total Distance: ', round(distance, 2))

In [ ]:
# BRKGA - Algorithm
parameters = {
            'population_size': 15,
            'elite': 1,
            'bias': 0.5,
            'mutants': 5,
            'generations': 3500,
            'verbose': True
             }
start = time.time()
route, distance = biased_random_key_genetic_algorithm(distance_matrix, **parameters)
end = time.time() - start

# Plot Locations and Tour
graphs.plot_tour(coordinates, city_tour = route, view = 'notebook', size = 10)
route = [x - 1 for x in route]
times_info["biased_random_key_genetic_algorithm"] = end
shortest_paths["biased_random_key_genetic_algorithm"] = f"Минимальная длина маршрута: {distance}, Путь: {route}"
print('Total Distance: ', round(distance, 2))

In [ ]:
# CI - Algorithm
parameters = {
            'verbose': True
             }
start = time.time()
route, distance = cheapest_insertion(distance_matrix, **parameters)
end = time.time() - start

# Plot Locations and Tour
graphs.plot_tour(coordinates, city_tour = route, view = 'notebook', size = 10)
route = [x - 1 for x in route]
times_info["cheapest_insertion"] = end
shortest_paths["cheapest_insertion"] = f"Минимальная длина маршрута: {distance}, Путь: {route}"
print('Total Distance: ', round(distance, 2))

In [ ]:
# CW - Parameters
parameters = {
            'local_search': True,
            'verbose': True
             }

# CW - Algorithm
start = time.time()
route, distance = clarke_wright_savings(distance_matrix, **parameters)
end = time.time() - start

# Plot Locations and Tour
graphs.plot_tour(coordinates, city_tour = route, view = 'notebook', size = 10)
route = [x - 1 for x in route]
times_info["clarke_wright_savings"] = end
shortest_paths["clarke_wright_savings"] = f"Минимальная длина маршрута: {distance}, Путь: {route}"
print('Total Distance: ', round(distance, 2))

In [ ]:
# Concave Hull Algorithm
start = time.time()
route, distance = concave_hull_algorithm(coordinates, distance_matrix, **parameters)
end = time.time() - start

# Plot Locations and Tour
graphs.plot_tour(coordinates, city_tour = route, view = 'notebook', size = 10)
route = [x - 1 for x in route]
times_info["concave_hull_algorithm"] = end
shortest_paths["concave_hull_algorithm"] = f"Минимальная длина маршрута: {distance}, Путь: {route}"
print('Total Distance: ', round(distance, 2))

In [ ]:
# Convex Hull Algorithm- Algorithm
start = time.time()
route, distance = convex_hull_algorithm(coordinates, distance_matrix, **parameters)
end = time.time() - start

# Plot Locations and Tour
graphs.plot_tour(coordinates, city_tour = route, view = 'notebook', size = 10)
route = [x - 1 for x in route]
times_info["convex_hull_algorithm"] = end
shortest_paths["convex_hull_algorithm"] = f"Минимальная длина маршрута: {distance}, Путь: {route}"
print('Total Distance: ', round(distance, 2))

In [ ]:
# Elastic Net - Parameters
parameters = {
            'alpha': 0.2,
            'beta': 2.0,
            'k': 0.2,
            'learning_rate': 0.75,
            'learning_upt': 25,
            'n_neurons': 100,
            'radius': 0.1,
            'iterations': 5000,
            'local_search': False,
            'verbose': True
             }

# Elastic Net - Algorithm
start = time.time()
route, distance = elastic_net_tsp(coordinates, distance_matrix, **parameters)
end = time.time() - start

# Plot Locations and Tour
graphs.plot_tour(coordinates, city_tour = route, view = 'notebook', size = 10)
route = [x - 1 for x in route]
times_info["elastic_net_tsp"] = end
shortest_paths["elastic_net_tsp"] = f"Минимальная длина маршрута: {distance}, Путь: {route}"
print('Total Distance: ', round(distance, 2))

In [ ]:
# EO - Parameters
parameters = {
              'iterations': 2500,
              'tau':1.8,
              'verbose': True
             }

# EO - Algorithm
start = time.time()
route, distance = extremal_optimization(distance_matrix, seed, **parameters)
end = time.time() - start

# Plot Locations and Tour
graphs.plot_tour(coordinates, city_tour = route, view = 'notebook', size = 10)
route = [x - 1 for x in route]
times_info["extremal_optimization"] = end
shortest_paths["extremal_optimization"] = f"Минимальная длина маршрута: {distance}, Путь: {route}"
print('Total Distance: ', round(distance, 2))

In [ ]:
# FI - Parameters
parameters = {
            'initial_location': -1, # -1 =  Try All Locations.
            'verbose': True
             }

# FI - Algorithm
start = time.time()
route, distance = farthest_insertion(distance_matrix, **parameters)
end = time.time() - start

# Plot Locations and Tour
graphs.plot_tour(coordinates, city_tour = route, view = 'notebook', size = 10)
route = [x - 1 for x in route]
times_info["farthest_insertion"] = end
shortest_paths["farthest_insertion"] = f"Минимальная длина маршрута: {distance}, Путь: {route}"
print('Total Distance: ', round(distance, 2))

In [ ]:
# GRASP - Parameters
parameters = {
              'iterations': 15,
              'rcl': 25,
              'greediness_value': 0.5,
              'verbose': True
             }

# GRASP - Algorithm
start = time.time()
route, distance = greedy_randomized_adaptive_search_procedure(distance_matrix, seed, **parameters)
end = time.time() - start

# Plot Locations and Tour
graphs.plot_tour(coordinates, city_tour = route, view = 'notebook', size = 10)
route = [x - 1 for x in route]
times_info["greedy_randomized_adaptive_search_procedure"] = end
shortest_paths["greedy_randomized_adaptive_search_procedure"] = f"Минимальная длина маршрута: {distance}, Путь: {route}"
print('Total Distance: ', round(distance, 2))

In [ ]:
# GKSP - Algorithm
parameters = {
            'verbose': True
             }
start = time.time()
route, distance = greedy_karp_steele_patching(distance_matrix, **parameters)
end = time.time() - start

# Plot Locations and Tour
graphs.plot_tour(coordinates, city_tour = route, view = 'notebook', size = 10)
route = [x - 1 for x in route]
times_info["greedy_karp_steele_patching"] = end
shortest_paths["greedy_karp_steele_patching"] = f"Минимальная длина маршрута: {distance}, Путь: {route}"
print('Total Distance: ', round(distance, 2))

In [ ]:
# GS - Parameters
parameters = {
              'alpha': 0.3,
              'iterations': 5000,
              'local_search_optima': 1000,
              'max_attempts': 100,
              'verbose': True
             }

# GS - Algorithm
start = time.time()
route, distance = guided_search(distance_matrix, seed, **parameters)
end = time.time() - start

# Plot Locations and Tour
graphs.plot_tour(coordinates, city_tour = route, view = 'notebook', size = 10)
route = [x - 1 for x in route]
times_info["guided_search"] = end
shortest_paths["guided_search"] = f"Минимальная длина маршрута: {distance}, Путь: {route}"
print('Total Distance: ', round(distance, 2))

In [ ]:
# HPN - Parameters
parameters = {
            'alpha': 50,
            'sigma': 1,
            'A': 100,
            'B': 100,
            'C': 90,
            'D': 100,
            'trials': 25,
            'iterations': 1500,
            'local_search': True,
            'verbose': True
             }

# HPN
start = time.time()
route, distance = hopfield_network_tsp(distance_matrix, **parameters)
end = time.time() - start

# Plot Locations and Tour
graphs.plot_tour(coordinates, city_tour = route, view = 'notebook', size = 10)
route = [x - 1 for x in route]
times_info["hopfield_network_tsp"] = end
shortest_paths["hopfield_network_tsp"] = f"Минимальная длина маршрута: {distance}, Путь: {route}"
print('Total Distance: ', round(distance, 2))

In [ ]:
# IS - Parameters
parameters = {
              'iterations': 10,
              'max_attempts':75000,
              'verbose': True
             }

# IS - Algorithm
start = time.time()
route, distance = iterated_search(distance_matrix, seed, **parameters)
end = time.time() - start

# Plot Locations and Tour
graphs.plot_tour(coordinates, city_tour = route, view = 'notebook', size = 10)
route = [x - 1 for x in route]
times_info["iterated_search"] = end
shortest_paths["iterated_search"] = f"Минимальная длина маршрута: {distance}, Путь: {route}"
print('Total Distance: ', round(distance, 2))

In [ ]:
# KSP - Algorithm

parameters = {
            'verbose': True
             }
start = time.time()
route, distance = karp_steele_patching(distance_matrix, **parameters)
end = time.time() - start

# Plot Locations and Tour
graphs.plot_tour(coordinates, city_tour = route, view = 'notebook', size = 10)
route = [x - 1 for x in route]
times_info["karp_steele_patching"] = end
shortest_paths["karp_steele_patching"] = f"Минимальная длина маршрута: {distance}, Путь: {route}"
print('Total Distance: ', round(distance, 2))

In [ ]:
# LNS - Parameters
parameters = {
              'iterations': 100,
              'neighborhood_size': 4,
              'local_search': True,
              'verbose': True
             }

# LNS - Algorithm
start = time.time()
route, distance = large_neighborhood_search(distance_matrix, **parameters)
end = time.time() - start

# Plot Locations and Tour
graphs.plot_tour(coordinates, city_tour = route, view = 'notebook', size = 10)
route = [x - 1 for x in route]
times_info["large_neighborhood_search"] = end
shortest_paths["large_neighborhood_search"] = f"Минимальная длина маршрута: {distance}, Путь: {route}"
print('Total Distance: ', round(distance, 2))

In [ ]:
# MF - Parameters
parameters = {
            'local_search': True,
            'verbose': True
             }

# MF - Algorithm
start = time.time()
route, distance = multifragment_heuristic(distance_matrix, **parameters)
end = time.time() - start

# Plot Locations and Tour
graphs.plot_tour(coordinates, city_tour = route, view = 'notebook', size = 10)
route = [x - 1 for x in route]
times_info["multifragment_heuristic"] = end
shortest_paths["multifragment_heuristic"] = f"Минимальная длина маршрута: {distance}, Путь: {route}"
print('Total Distance: ', round(distance, 2))

In [ ]:
# NI - Parameters
parameters = {
            'initial_location': -1, # -1 =  Try All Locations.
            'verbose': True
             }

# NI - Algorithm
start = time.time()
route, distance = nearest_insertion(distance_matrix, **parameters)
end = time.time() - start

# Plot Locations and Tour
graphs.plot_tour(coordinates, city_tour = route, view = 'notebook', size = 10)
route = [x - 1 for x in route]
times_info["nearest_insertion"] = end
shortest_paths["nearest_insertion"] = f"Минимальная длина маршрута: {distance}, Путь: {route}"
print('Total Distance: ', round(distance, 2))

In [ ]:
# NN - Parameters
parameters = {
            'initial_location': -1, # -1 =  Try All Locations.
            'local_search': True,
            'verbose': True
             }

# NN - Algorithm
start = time.time()
route, distance = nearest_neighbour(distance_matrix, **parameters)
end = time.time() - start

# Plot Locations and Tour
graphs.plot_tour(coordinates, city_tour = route, view = 'notebook', size = 10)
route = [x - 1 for x in route]
times_info["nearest_neighbour_pyComb"] = end
shortest_paths["nearest_neighbour_pyComb"] = f"Минимальная длина маршрута: {distance}, Путь: {route}"
print('Total Distance: ', round(distance, 2))

In [ ]:
# RI - Algorithm

parameters = {
            'initial_location': -1, # -1 =  Try All Locations.
            'verbose': True
             }
start = time.time()
route, distance = random_insertion(distance_matrix, **parameters)
end = time.time() - start

# Plot Locations and Tour
graphs.plot_tour(coordinates, city_tour = route, view = 'notebook', size = 10)
route = [x - 1 for x in route]
times_info["random_insertion"] = end
shortest_paths["random_insertion"] = f"Минимальная длина маршрута: {distance}, Путь: {route}"
print('Total Distance: ', round(distance, 2))

In [ ]:
# RT - Parameters
parameters = {
            'search': 25,
            'local_search': True,
            'verbose': True
             }

# RT - Algorithm
start = time.time()
route, distance = random_tour(distance_matrix, **parameters)
end = time.time() - start

# Plot Locations and Tour
graphs.plot_tour(coordinates, city_tour = route, view = 'notebook', size = 10)
route = [x - 1 for x in route]
times_info["random_tour"] = end
shortest_paths["random_tour"] = f"Минимальная длина маршрута: {distance}, Путь: {route}"
print('Total Distance: ', round(distance, 2))

In [ ]:
# SS - Parameters
parameters = {
              'iterations': 150,
              'reference_size': 10,
              'reverse_prob': 0.5,
              'scramble_prob': 0.3,
              'verbose': True
             }

# SS - Algortihm
start = time.time()
route, distance = scatter_search(distance_matrix, seed, **parameters)
end = time.time() - start

# Plot Locations and Tour
graphs.plot_tour(coordinates, city_tour = route, view = 'notebook', size = 10)
route = [x - 1 for x in route]
times_info["scatter_search"] = end
shortest_paths["scatter_search"] = f"Минимальная длина маршрута: {distance}, Путь: {route}"
print('Total Distance: ', round(distance, 2))

In [ ]:
# SA - Parameters
parameters = {
            'initial_temperature': 1.0,
            'temperature_iterations': 10,
            'final_temperature': 0.0001,
            'alpha': 0.9,
            'verbose': True
             }

# SA
start = time.time()
route, distance = simulated_annealing_tsp(distance_matrix, **parameters)
end = time.time() - start

# Plot Locations and Tour
graphs.plot_tour(coordinates, city_tour = route, view = 'notebook', size = 10)
route = [x - 1 for x in route]
times_info["simulated_annealing_pyComb"] = end
shortest_paths["simulated_annealing_pyComb"] = f"Минимальная длина маршрута: {distance}, Путь: {route}"
print('Total Distance: ', round(distance, 2))

In [ ]:
# SOM - Parameters
parameters = {
            'size_multiplier': 4,
            'iterations': 25000,
            'decay_nr': 0.99997,
            'decay_lr': 0.99997,
            'learning_rate': 0.80,
            'local_search': True,
            'verbose': True
             }

# SOM - Algorithm
start = time.time()
route, distance = self_organizing_maps(coordinates, distance_matrix, **parameters)
end = time.time() - start

# Plot Locations and Tour
graphs.plot_tour(coordinates, city_tour = route, view = 'notebook', size = 10)
route = [x - 1 for x in route]
times_info["self_organizing_maps"] = end
shortest_paths["self_organizing_maps"] = f"Минимальная длина маршрута: {distance}, Путь: {route}"
print('Total Distance: ', round(distance, 2))

In [ ]:
# SFC (Hilbert)  - Parameters
parameters = {
            'local_search': True,
            'verbose': True
             }

# SFC (Hilbert)  Algorithm
start = time.time()
route, distance = space_filling_curve_h(coordinates, distance_matrix, **parameters)
end = time.time() - start

# Plot Locations and Tour
graphs.plot_tour(coordinates, city_tour = route, view = 'notebook', size = 10)
route = [x - 1 for x in route]
times_info["space_filling_curve_h"] = end
shortest_paths["space_filling_curve_h"] = f"Минимальная длина маршрута: {distance}, Путь: {route}"
print('Total Distance: ', round(distance, 2))

In [ ]:

start = time.time()
route, distance = space_filling_curve_m(coordinates, distance_matrix, **parameters)
end = time.time() - start

# Plot Locations and Tour
graphs.plot_tour(coordinates, city_tour = route, view = 'notebook', size = 10)
route = [x - 1 for x in route]
times_info["space_filling_curve_m"] = end
shortest_paths["space_filling_curve_m"] = f"Минимальная длина маршрута: {distance}, Путь: {route}"
print('Total Distance: ', round(distance, 2))

In [ ]:

start = time.time()
route, distance = space_filling_curve_s(coordinates, distance_matrix, **parameters)
end = time.time() - start

# Plot Locations and Tour
graphs.plot_tour(coordinates, city_tour = route, view = 'notebook', size = 10)
route = [x - 1 for x in route]
times_info["space_filling_curve_s"] = end
shortest_paths["space_filling_curve_s"] = f"Минимальная длина маршрута: {distance}, Путь: {route}"
print('Total Distance: ', round(distance, 2))

In [ ]:
# SHC - Parameters
parameters = {
              'iterations': 500,
              'verbose': True
             }

# SHC - Algorithm
start = time.time()
route, distance = stochastic_hill_climbing(distance_matrix, seed, **parameters)
end = time.time() - start

# Plot Locations and Tour
graphs.plot_tour(coordinates, city_tour = route, view = 'notebook', size = 10)
route = [x - 1 for x in route]
times_info["stochastic_hill_climbing"] = end
shortest_paths["stochastic_hill_climbing"] = f"Минимальная длина маршрута: {distance}, Путь: {route}"
print('Total Distance: ', round(distance, 2))

In [ ]:
# SW - Parameters
parameters = {
            'initial_location': -1, # -1 =  Try All Locations.
            'verbose': True
             }

# SW - Algorithm
start = time.time()
route, distance = sweep(coordinates, distance_matrix, **parameters)
end = time.time() - start

# Plot Locations and Tour
graphs.plot_tour(coordinates, city_tour = route, view = 'notebook', size = 10)
route = [x - 1 for x in route]
times_info["sweep"] = end
shortest_paths["sweep"] = f"Минимальная длина маршрута: {distance}, Путь: {route}"
print('Total Distance: ', round(distance, 2))

In [ ]:
# TS - Parameters
parameters = {
              'iterations': 500,
              'tabu_tenure': 75,
              'verbose': True
             }

# TS - Algorithm
start = time.time()
route, distance = tabu_search(distance_matrix, seed, **parameters)
end = time.time() - start

# Plot Locations and Tour
graphs.plot_tour(coordinates, city_tour = route, view = 'notebook', size = 10)
route = [x - 1 for x in route]
times_info["tabu_search_pyComb"] = end
shortest_paths["tabu_search_pyComb"] = f"Минимальная длина маршрута: {distance}, Путь: {route}"
print('Total Distance: ', round(distance, 2))

In [ ]:
# TBB - Algorithm
parameters = {
            'verbose': True
             }
start = time.time()
route, distance = truncated_branch_and_bound(distance_matrix, **parameters)
end = time.time() - start

# Plot Locations and Tour
graphs.plot_tour(coordinates, city_tour = route, view = 'notebook', size = 10)
route = [x - 1 for x in route]
times_info["truncated_branch_and_bound"] = end
shortest_paths["truncated_branch_and_bound"] = f"Минимальная длина маршрута: {distance}, Путь: {route}"
print('Total Distance: ', round(distance, 2))

In [ ]:
# VNS - Parameters
parameters = {
              'iterations': 50,
              'max_attempts': 50,
              'neighbourhood_size': 15,
              'verbose': True
             }

# VNS - Algorithm
start = time.time()
route, distance = variable_neighborhood_search(distance_matrix, seed, **parameters)
end = time.time() - start

# Plot Locations and Tour
graphs.plot_tour(coordinates, city_tour = route, view = 'notebook', size = 10)
route = [x - 1 for x in route]
times_info["variable_neighborhood_search"] = end
shortest_paths["variable_neighborhood_search"] = f"Минимальная длина маршрута: {distance}, Путь: {route}"
print('Total Distance: ', round(distance, 2))

# Гибридные методы

In [ ]:
# Нельзя сохранять промежуточное решение
import time
import numpy as np
import networkx as nx
from scipy.spatial.distance import pdist, squareform
from pulp import LpProblem, LpVariable, LpMinimize, lpSum, value

class TSP_Branch_Cut:
    """Branch & Cut solver for Traveling Salesman Problem."""

    def __init__(self, dist_matrix=None):
        """
        Initialize the TSP Branch & Cut solver.

        Args:
            dist_matrix: Distance matrix between nodes (optional)
        """
        self.dist_matrix = dist_matrix
        self.num_nodes = 0 if dist_matrix is None else len(dist_matrix)
        self.model = None
        self.edge_vars = {}

    def solve(self, **kwargs):
        """
        Solve the TSP using Branch & Cut method.

        Returns:
            tour: Optimal tour as list of node indices
            tour_length: Length of optimal tour
        """
        # Start timing
        start_time = time.time()

        # Use coords if provided, otherwise use existing dist_matrix
        if 'nodes_coord' in kwargs:
            nodes_coord = kwargs['nodes_coord']
            self.num_nodes = nodes_coord.shape[0]
            # Get distance matrix
            self.dist_matrix = squareform(pdist(nodes_coord, metric='euclidean'))

        if self.dist_matrix is None:
            raise ValueError("Either dist_matrix must be provided at initialization or nodes_coord must be provided to solve method")

        # Create ILP model
        self.model = LpProblem("TSP_BranchCut", LpMinimize)

        # Create edge variables - x[i,j] = 1 if edge (i,j) is used
        self._create_edge_variables()

        # Set objective function
        self._set_objective()

        # Add degree constraints
        self._add_degree_constraints()

        # Solve with subtour elimination
        tour, edges = self._solve_with_subtour_elimination()

        # Calculate tour length
        tour_length = self._calculate_tour_length(tour)

        # Calculate solve time
        solve_time = time.time() - start_time

        return tour, tour_length

    def _create_edge_variables(self):
        """Create binary variables for each edge in the graph."""
        self.edge_vars = {}
        for i in range(self.num_nodes):
            for j in range(i+1, self.num_nodes):  # Only need one direction for undirected graph
                self.edge_vars[i, j] = LpVariable(f'x_{i}_{j}', cat='Binary')

    def get_edge_var(self, i, j):
        """Helper function to access edge variable in either direction."""
        return self.edge_vars[min(i, j), max(i, j)]

    def _set_objective(self):
        """Set the objective function to minimize total distance."""
        self.model += lpSum([self.dist_matrix[i][j] * self.get_edge_var(i, j)
                            for i in range(self.num_nodes)
                            for j in range(i+1, self.num_nodes)])

    def _add_degree_constraints(self):
        """Add constraints that each node must have exactly two edges."""
        for i in range(self.num_nodes):
            self.model += lpSum([self.get_edge_var(i, j)
                                for j in range(self.num_nodes) if j != i]) == 2

    def _solve_with_subtour_elimination(self):
        """Solve the model with iterative subtour elimination."""
        edges = []
        G = nx.Graph()

        # Iteratively solve and add subtour elimination constraints
        while True:
            # Solve current model
            self.model.solve()

            # Extract solution edges
            edges = []
            for i in range(self.num_nodes):
                for j in range(i+1, self.num_nodes):
                    if value(self.get_edge_var(i, j)) > 0.5:
                        edges.append((i, j))

            # Build graph from edges
            G.clear()
            G.add_edges_from(edges)

            # Find connected components (subtours)
            components = list(nx.connected_components(G))

            if len(components) == 1:
                # Valid tour found
                break

            # Add subtour elimination constraints and resolve
            for component in components:
                if len(component) < self.num_nodes:
                    component = list(component)
                    self.model += lpSum([self.get_edge_var(i, j)
                                        for i in component
                                        for j in component if i < j]) <= len(component) - 1

        # Convert edge list to tour
        tour = self._build_tour_from_edges(G)

        return tour, edges

    def _build_tour_from_edges(self, G):
        """Build a tour from the graph edges."""
        tour = []
        current = 0  # Start from node 0
        tour.append(current)

        for _ in range(self.num_nodes-1):
            neighbors = list(G.neighbors(current))
            # Remove already visited nodes
            unvisited = [n for n in neighbors if n not in tour]
            if not unvisited:
                break
            current = unvisited[0]
            tour.append(current)

        return tour

    def _calculate_tour_length(self, tour):
        """Calculate the length of the tour."""
        tour_length = 0
        for i in range(len(tour)):
            j = (i + 1) % len(tour)
            tour_length += self.dist_matrix[tour[i]][tour[j]]

        return tour_length

In [ ]:
import numpy as np
import math
import random
import time

class TSP_2opt_SA_naive:
    def __init__(self, distance_matrix):
        """
        Initialize the solver with a distance matrix.

        Args:
            distance_matrix: 2D numpy array where distance_matrix[i][j] is the distance from city i to city j
        """
        self.distances = distance_matrix
        self.num_cities = len(distance_matrix)

    def calculate_tour_length(self, tour):
        """Calculate the total length of a tour."""
        length = 0
        for i in range(self.num_cities):
            length += self.distances[tour[i]][tour[(i + 1) % self.num_cities]]
        return length

    def two_opt_swap(self, tour, i, j):
        """Perform a 2-opt swap by reversing the segment between positions i and j."""
        if i > j:
            i, j = j, i
        new_tour = tour.copy()
        # Reverse the segment between i and j
        new_tour[i:j+1] = reversed(tour[i:j+1])
        return new_tour

    def get_random_neighbor(self, tour):
        """Get a random neighbor by performing a 2-opt swap between random positions."""
        i, j = sorted(random.sample(range(self.num_cities), 2))
        return self.two_opt_swap(tour, i, j)

    def acceptance_probability(self, old_cost, new_cost, temperature):
        """Calculate probability of accepting a worse solution based on the current temperature."""
        if new_cost < old_cost:
            return 1.0
        return math.exp((old_cost - new_cost) / temperature)

    def solve(self, initial_tour=None, initial_temp=100.0, cooling_rate=0.995,
              min_temp=1e-6, max_iterations=10000, max_iterations_without_improvement=1000):
        """
        Solve the TSP using 2-opt with Simulated Annealing.

        Args:
            initial_tour: Starting tour (random if None)
            initial_temp: Starting temperature for simulated annealing
            cooling_rate: Rate at which temperature decreases
            min_temp: Minimum temperature before stopping
            max_iterations: Maximum number of iterations
            max_iterations_without_improvement: Stop early if no improvement for this many iterations

        Returns:
            best_tour, best_length: The best tour found and its length
        """
        # Initialize tour randomly if not provided
        if initial_tour is None:
            current_tour = list(range(self.num_cities))
            random.shuffle(current_tour)
        else:
            current_tour = initial_tour.copy()

        # Initialize values
        current_length = self.calculate_tour_length(current_tour)
        best_tour = current_tour.copy()
        best_length = current_length

        temperature = initial_temp
        iterations_without_improvement = 0

        start_time = time.time()

        # Main loop
        for iteration in range(max_iterations):
            # Get a random neighbor by performing a 2-opt swap
            neighbor_tour = self.get_random_neighbor(current_tour)
            neighbor_length = self.calculate_tour_length(neighbor_tour)

            # Decide whether to accept the new solution
            if self.acceptance_probability(current_length, neighbor_length, temperature) > random.random():
                current_tour = neighbor_tour
                current_length = neighbor_length

                # Update best solution if improved
                if current_length < best_length:
                    best_tour = current_tour.copy()
                    best_length = current_length
                    iterations_without_improvement = 0
                else:
                    iterations_without_improvement += 1
            else:
                iterations_without_improvement += 1

            # Cool down
            temperature *= cooling_rate

            # Termination conditions
            if temperature < min_temp or iterations_without_improvement >= max_iterations_without_improvement:
                break

        solve_time = time.time() - start_time
        print(f"Solved in {iteration+1} iterations, {solve_time:.2f} seconds")
        print(f"Best tour length: {best_length:.2f}")

        return best_tour, best_length

    def improve_tour_with_2opt(self, tour, max_iterations=1000):
        """
        Apply 2-opt local search to improve a tour.

        Args:
            tour: Initial tour to improve
            max_iterations: Maximum number of iterations

        Returns:
            Improved tour and its length
        """
        best_tour = tour.copy()
        best_length = self.calculate_tour_length(best_tour)
        improvement = True
        iterations = 0

        while improvement and iterations < max_iterations:
            improvement = False
            iterations += 1

            for i in range(self.num_cities - 2):
                for j in range(i + 2, self.num_cities):
                    # Skip if we would be reversing the whole tour (equivalent to original)
                    if i == 0 and j == self.num_cities - 1:
                        continue

                    # Try the 2-opt swap
                    new_tour = self.two_opt_swap(best_tour, i, j)
                    new_length = self.calculate_tour_length(new_tour)

                    # Accept if improved
                    if new_length < best_length:
                        best_tour = new_tour
                        best_length = new_length
                        improvement = True
                        break

                if improvement:
                    break

        return best_tour, best_length

    def solve_hybrid(self, initial_tour=None, sa_params=None, local_search_after_sa=True):
        """
        Hybrid approach: Run simulated annealing first, then improve with deterministic 2-opt.

        Args:
            initial_tour: Starting tour (random if None)
            sa_params: Dictionary of parameters for simulated annealing
            local_search_after_sa: Whether to apply deterministic 2-opt after SA

        Returns:
            best_tour, best_length: The best tour found and its length
        """
        # Set default SA parameters
        if sa_params is None:
            sa_params = {
                'initial_temp': 100.0,
                'cooling_rate': 0.995,
                'min_temp': 1e-6,
                'max_iterations': 5000,
                'max_iterations_without_improvement': 500
            }

        # Run simulated annealing
        best_tour, best_length = self.solve(
            initial_tour=initial_tour,
            initial_temp=sa_params['initial_temp'],
            cooling_rate=sa_params['cooling_rate'],
            min_temp=sa_params['min_temp'],
            max_iterations=sa_params['max_iterations'],
            max_iterations_without_improvement=sa_params['max_iterations_without_improvement']
        )

        # Apply deterministic 2-opt local search to further improve
        if local_search_after_sa:
            print("Applying deterministic 2-opt local search...")
            best_tour, best_length = self.improve_tour_with_2opt(best_tour)
            print(f"Final tour length after local search: {best_length:.2f}")

        return best_tour, best_length

In [ ]:
import numpy as np
import math
import random
import time

class TSP_2opt_SA_outer:
    def __init__(self, distance_matrix):
        """
        Initialize the solver with a distance matrix.

        Args:
            distance_matrix: 2D numpy array where distance_matrix[i][j] is the distance from city i to city j
        """
        self.distances = distance_matrix
        self.num_cities = len(distance_matrix)

    def calculate_tour_length(self, tour):
        """Calculate the total length of a tour."""
        return sum(self.distances[tour[i]][tour[(i + 1) % self.num_cities]] for i in range(self.num_cities))

    def two_opt_swap(self, tour, i, j):
        """
        Perform a 2-opt swap by reversing the segment between positions i and j.
        Returns a new tour.
        """
        new_tour = tour.copy()
        new_tour[i:j+1] = new_tour[i:j+1][::-1]
        return new_tour

    def delta_two_opt(self, tour, i, j):
        """
        Calculate the cost difference (delta) for a 2-opt swap between positions i and j in O(1) time.
        """
        n = self.num_cities
        a, b = tour[i - 1], tour[i]
        c, d = tour[j], tour[(j + 1) % n]
        delta = (self.distances[a][c] + self.distances[b][d]) - (self.distances[a][b] + self.distances[c][d])
        return delta

    def get_random_neighbor(self, tour):
        """
        Get a random neighbor by performing a 2-opt swap between two random indices.
        Returns the new tour and the cost difference (delta).
        """
        i, j = sorted(random.sample(range(self.num_cities), 2))
        new_tour = self.two_opt_swap(tour, i, j)
        delta = self.delta_two_opt(tour, i, j)
        return new_tour, delta, i, j

    def acceptance_probability(self, delta, temperature):
        """
        Calculate the acceptance probability.
        Returns 1.0 for improvements.
        """
        return 1.0 if delta < 0 else math.exp(-delta / temperature)

    def double_bridge_move(self, tour):
        """
        Perform a double-bridge move to perturb the current tour.
        This move breaks the tour into four segments and rearranges them.
        """
        n = self.num_cities
        pos1 = random.randint(1, n // 4)
        pos2 = random.randint(pos1 + 1, n // 2)
        pos3 = random.randint(pos2 + 1, 3 * n // 4)
        new_tour = tour[0:pos1] + tour[pos3:] + tour[pos2:pos3] + tour[pos1:pos2]
        return new_tour

    def solve_sa(self, initial_tour, sa_params, use_perturbation=True):
        """
        Run the SA phase from a given starting tour.
        Returns the best tour, its length, number of iterations, and elapsed time.
        """
        current_tour = initial_tour.copy()
        current_length = self.calculate_tour_length(current_tour)
        best_tour, best_length = current_tour.copy(), current_length

        temperature = sa_params.get('initial_temp', 500.0)
        cooling_rate = sa_params.get('cooling_rate', 0.9995)
        min_temp = sa_params.get('min_temp', 1e-6)
        max_iterations = sa_params.get('max_iterations', 10000)
        max_no_improve = sa_params.get('max_no_improve', 1000)
        perturb_interval = sa_params.get('perturb_interval', 500)
        perturb_prob = sa_params.get('perturb_prob', 0.1)

        iterations = 0
        no_improve = 0
        start_time = time.time()

        while iterations < max_iterations and temperature > min_temp and no_improve < max_no_improve:
            iterations += 1
            # Optionally apply a perturbation
            if use_perturbation and iterations % perturb_interval == 0 and random.random() < perturb_prob:
                perturbed = self.double_bridge_move(current_tour)
                perturbed_length = self.calculate_tour_length(perturbed)
                # Accept perturbation if it improves or with small probability
                if perturbed_length < current_length or random.random() < 0.2:
                    current_tour = perturbed
                    current_length = perturbed_length
                    no_improve = 0

            neighbor, delta, i_swap, j_swap = self.get_random_neighbor(current_tour)
            if self.acceptance_probability(delta, temperature) > random.random():
                current_tour = neighbor
                current_length += delta
                if current_length < best_length:
                    best_tour = current_tour.copy()
                    best_length = current_length
                    no_improve = 0
                else:
                    no_improve += 1
            else:
                no_improve += 1
            temperature *= cooling_rate

        elapsed = time.time() - start_time
        return best_tour, best_length, iterations, elapsed

    def improve_tour_with_2opt(self, tour, max_iterations=500):
        """
        Run deterministic 2-opt local search.
        Returns the improved tour, its length, and the number of iterations.
        """
        best_tour = tour.copy()
        best_length = self.calculate_tour_length(best_tour)
        iterations = 0
        improved = True

        while improved and iterations < max_iterations:
            improved = False
            iterations += 1
            for i in range(1, self.num_cities - 1):
                for j in range(i + 1, self.num_cities):
                    if i == 1 and j == self.num_cities - 1:
                        continue  # Avoid trivial full reversal
                    delta = self.delta_two_opt(best_tour, i, j)
                    if delta < 0:
                        best_tour = self.two_opt_swap(best_tour, i, j)
                        best_length += delta
                        improved = True
                        break
                if improved:
                    break
        return best_tour, best_length, iterations

    def solve(self, outer_iterations=10, sa_params=None, local_search_after_sa=True, use_perturbation=True):
        """
        Hybrid iterated approach: repeatedly run SA followed by (optional) 2-opt.
        This iterated local search allows restarting from perturbed solutions.

        Args:
            outer_iterations: Number of outer iterations.
            sa_params: Parameters for the SA phase.
            local_search_after_sa: Whether to perform 2-opt after SA.
            use_perturbation: Whether to use the double-bridge perturbation in SA.

        Returns:
            overall_best_tour, overall_best_length: Best solution found.
        """
        if sa_params is None:
            sa_params = {
                'initial_temp': 500.0,
                'cooling_rate': 0.9995,
                'min_temp': 1e-6,
                'max_iterations': 10000,
                'max_no_improve': 1000,
                'perturb_interval': 500,
                'perturb_prob': 0.1
            }

        overall_best_tour = None
        overall_best_length = float('inf')
        overall_times = []

        # Start with a random tour
        current_tour = list(range(self.num_cities))
        random.shuffle(current_tour)

        for outer in range(1, outer_iterations + 1):
            outer_start = time.time()
            print(f"Outer iteration {outer}/{outer_iterations}")
            # Run SA from the current tour
            sa_tour, sa_length, sa_iters, sa_time = self.solve_sa(current_tour, sa_params, use_perturbation)
            print(f"  SA: {sa_iters} iterations, {sa_time:.4f} seconds, tour length: {sa_length:.2f}")

            # Apply 2-opt improvement if enabled
            if local_search_after_sa:
                print("  Applying deterministic 2-opt local search...")
                opt_tour, opt_length, opt_iters = self.improve_tour_with_2opt(sa_tour)
                print(f"  2-opt: {opt_iters} iterations, final tour length: {opt_length:.2f}")
                candidate_length = opt_length
                candidate_tour = opt_tour
            else:
                candidate_length = sa_length
                candidate_tour = sa_tour

            total_outer = time.time() - outer_start
            print(f"  Outer iteration time: {total_outer:.4f} seconds")

            # Update overall best if found
            if candidate_length < overall_best_length:
                overall_best_length = candidate_length
                overall_best_tour = candidate_tour.copy()
                print(f"  New best solution found: {overall_best_length:.2f}")
            # Use the candidate as starting point for next outer iteration
            current_tour = candidate_tour.copy()
            overall_times.append(total_outer)
            print("")

        avg_time = sum(overall_times)/len(overall_times) if overall_times else 0.0
        print(f"Best solution overall: {overall_best_length:.2f}")
        print(f"Mean outer iteration time: {avg_time:.4f} seconds")
        return overall_best_tour, overall_best_length


In [ ]:
import numpy as np
import math
import random
import time

class TSP_2opt_3opt_SA:
    def __init__(self, distance_matrix):
        """
        Initialize the solver with a distance matrix.

        Args:
            distance_matrix: 2D numpy array where distance_matrix[i][j] is the distance from city i to city j
        """
        self.distances = distance_matrix
        self.num_cities = len(distance_matrix)

    def calculate_tour_length(self, tour):
        """Calculate the total length of a tour."""
        return sum(self.distances[tour[i]][tour[(i + 1) % self.num_cities]] for i in range(self.num_cities))

    def two_opt_swap(self, tour, i, j):
        """
        Perform a 2-opt swap by reversing the segment between positions i and j.
        Returns a new tour.
        """
        new_tour = tour.copy()
        new_tour[i:j+1] = new_tour[i:j+1][::-1]
        return new_tour

    def delta_two_opt(self, tour, i, j):
        """
        Calculate the cost difference (delta) for a 2-opt swap between positions i and j in O(1) time.
        """
        n = self.num_cities
        a, b = tour[i - 1], tour[i]
        c, d = tour[j], tour[(j + 1) % n]
        delta = (self.distances[a][c] + self.distances[b][d]) - (self.distances[a][b] + self.distances[c][d])
        return delta

    def get_random_neighbor(self, tour):
        """
        Get a random neighbor by performing a 2-opt swap between two random indices.
        Returns the new tour, the cost difference (delta), and the indices used.
        """
        i, j = sorted(random.sample(range(self.num_cities), 2))
        new_tour = self.two_opt_swap(tour, i, j)
        delta = self.delta_two_opt(tour, i, j)
        return new_tour, delta, i, j

    def acceptance_probability(self, delta, temperature):
        """
        Calculate the acceptance probability.
        Returns 1.0 for improvements.
        """
        return 1.0 if delta < 0 else math.exp(-delta / temperature)

    def double_bridge_move(self, tour):
        """
        Perform a double-bridge move to perturb the current tour.
        This move breaks the tour into four segments and rearranges them.
        """
        n = self.num_cities
        pos1 = random.randint(1, n // 4)
        pos2 = random.randint(pos1 + 1, n // 2)
        pos3 = random.randint(pos2 + 1, 3 * n // 4)
        new_tour = tour[0:pos1] + tour[pos3:] + tour[pos2:pos3] + tour[pos1:pos2]
        return new_tour

    def solve_sa(self, initial_tour, sa_params, use_perturbation=True):
        """
        Run the SA phase from a given starting tour.
        Returns the best tour, its length, number of iterations, and elapsed time.
        """
        current_tour = initial_tour.copy()
        current_length = self.calculate_tour_length(current_tour)
        best_tour, best_length = current_tour.copy(), current_length

        temperature = sa_params.get('initial_temp', 500.0)
        cooling_rate = sa_params.get('cooling_rate', 0.9995)
        min_temp = sa_params.get('min_temp', 1e-6)
        max_iterations = sa_params.get('max_iterations', 10000)
        max_no_improve = sa_params.get('max_no_improve', 1000)
        perturb_interval = sa_params.get('perturb_interval', 500)
        perturb_prob = sa_params.get('perturb_prob', 0.1)

        iterations = 0
        no_improve = 0
        start_time = time.time()

        # Counter for periodic recalculation
        recalc_interval = 100

        while iterations < max_iterations and temperature > min_temp and no_improve < max_no_improve:
            iterations += 1

            # Optionally apply a perturbation
            if use_perturbation and iterations % perturb_interval == 0 and random.random() < perturb_prob:
                perturbed = self.double_bridge_move(current_tour)
                perturbed_length = self.calculate_tour_length(perturbed)
                if perturbed_length < current_length or random.random() < 0.2:
                    current_tour = perturbed
                    current_length = perturbed_length
                    no_improve = 0

            neighbor, delta, i_swap, j_swap = self.get_random_neighbor(current_tour)
            if self.acceptance_probability(delta, temperature) > random.random():
                current_tour = neighbor
                current_length += delta
                # Every so often recalc the current length to prevent drift
                if iterations % recalc_interval == 0:
                    current_length = self.calculate_tour_length(current_tour)
                if current_length < best_length:
                    best_tour = current_tour.copy()
                    best_length = current_length
                    no_improve = 0
                else:
                    no_improve += 1
            else:
                no_improve += 1
            temperature *= cooling_rate

        elapsed = time.time() - start_time
        return best_tour, best_length, iterations, elapsed

    def improve_tour_with_2opt(self, tour, max_iterations=500):
        """
        Run deterministic 2-opt local search.
        Returns the improved tour, its length, and the number of iterations.
        """
        best_tour = tour.copy()
        best_length = self.calculate_tour_length(best_tour)
        iterations = 0
        improved = True

        while improved and iterations < max_iterations:
            improved = False
            iterations += 1
            for i in range(1, self.num_cities - 1):
                for j in range(i + 1, self.num_cities):
                    if i == 1 and j == self.num_cities - 1:
                        continue  # Avoid trivial full reversal
                    delta = self.delta_two_opt(best_tour, i, j)
                    if delta < 0:
                        best_tour = self.two_opt_swap(best_tour, i, j)
                        best_length += delta
                        improved = True
                        break
                if improved:
                    break
        return best_tour, best_length, iterations

    def improve_tour_with_3opt(self, tour, max_iterations=50, sample_fraction=0.5):
        """
        Run a more efficient deterministic 3-opt local search.
        Instead of checking all possible triplets (which is O(n^3)), we randomly sample a fraction of them.

        Args:
            tour: The starting tour.
            max_iterations: Maximum iterations to perform.
            sample_fraction: Fraction (0 to 1) of the triplets to consider. Lowering this speeds up 3-opt.

        Returns:
            improved tour, its length, and the number of iterations.
        """
        best_tour = tour.copy()
        best_length = self.calculate_tour_length(best_tour)
        iterations = 0

        # Pre-compute all valid triplet indices for the full search
        all_triplets = [(i, j, k) for i in range(1, self.num_cities - 2)
                        for j in range(i + 1, self.num_cities - 1)
                        for k in range(j + 1, self.num_cities)]
        total_triplets = len(all_triplets)

        improved = True
        while improved and iterations < max_iterations:
            improved = False
            iterations += 1

            # Sample a subset of triplets to consider
            sample_size = max(1, int(sample_fraction * total_triplets))
            sampled_triplets = random.sample(all_triplets, sample_size)

            for (i, j, k) in sampled_triplets:
                # Create candidate tours using different reconnections
                candA = best_tour[:i] + best_tour[j:k] + best_tour[i:j] + best_tour[k:]
                candB = best_tour[:i] + best_tour[j:k] + best_tour[i:j][::-1] + best_tour[k:]
                candC = best_tour[:i] + best_tour[j:k][::-1] + best_tour[i:j] + best_tour[k:]
                candD = best_tour[:i] + best_tour[j:k][::-1] + best_tour[i:j][::-1] + best_tour[k:]

                for candidate in [candA, candB, candC, candD]:
                    cand_length = self.calculate_tour_length(candidate)
                    if cand_length < best_length:
                        best_tour = candidate
                        best_length = cand_length
                        improved = True
                        break
                if improved:
                    break  # exit early if an improvement is found
        return best_tour, best_length, iterations

    def solve(self, outer_iterations=10, sa_params=None, local_search_after_sa=True, use_3opt=True, use_perturbation=True):
        """
        Hybrid iterated approach: repeatedly run SA followed by (optional) 2-opt and 3-opt local search.
        This iterated local search allows restarting from perturbed solutions.

        Args:
            outer_iterations: Number of outer iterations.
            sa_params: Parameters for the SA phase.
            local_search_after_sa: Whether to perform 2-opt after SA.
            use_3opt: Whether to run an additional 3-opt phase after 2-opt.
            use_perturbation: Whether to use the double-bridge perturbation in SA.

        Returns:
            overall_best_tour, overall_best_length: Best solution found.
        """
        if sa_params is None:
            sa_params = {
                'initial_temp': 500.0,
                'cooling_rate': 0.9995,
                'min_temp': 1e-6,
                'max_iterations': 10000,
                'max_no_improve': 1000,
                'perturb_interval': 500,
                'perturb_prob': 0.1
            }

        overall_best_tour = None
        overall_best_length = float('inf')
        overall_times = []

        # Start with a random tour
        current_tour = list(range(self.num_cities))
        random.shuffle(current_tour)

        for outer in range(1, outer_iterations + 1):
            outer_start = time.time()
            print(f"Outer iteration {outer}/{outer_iterations}")
            # Run SA phase
            sa_tour, sa_length, sa_iters, sa_time = self.solve_sa(current_tour, sa_params, use_perturbation)
            print(f"  SA: {sa_iters} iterations, {sa_time:.4f} sec, tour length: {sa_length:.2f}")

            # Apply 2-opt improvement if enabled
            if local_search_after_sa:
                print("  Applying deterministic 2-opt local search...")
                opt_tour, opt_length, opt_iters = self.improve_tour_with_2opt(sa_tour)
                print(f"  2-opt: {opt_iters} iterations, tour length: {opt_length:.2f}")
                candidate_tour = opt_tour
                candidate_length = opt_length
                # Optionally apply 3-opt improvement
                if use_3opt:
                    print("  Applying deterministic 3-opt local search...")
                    three_tour, three_length, three_iters = self.improve_tour_with_3opt(candidate_tour)
                    print(f"  3-opt: {three_iters} iterations, tour length: {three_length:.2f}")
                    candidate_tour = three_tour
                    candidate_length = three_length
            else:
                candidate_tour = sa_tour
                candidate_length = sa_length

            total_outer = time.time() - outer_start
            print(f"  Outer iteration time: {total_outer:.4f} sec")

            # Update overall best if found
            if candidate_length < overall_best_length:
                overall_best_length = candidate_length
                overall_best_tour = candidate_tour.copy()
                print(f"  New best solution found: {overall_best_length:.2f}")
            # Use the candidate as the starting point for the next iteration
            current_tour = candidate_tour.copy()
            overall_times.append(total_outer)
            print("")

        avg_time = sum(overall_times) / len(overall_times) if overall_times else 0.0
        print(f"Best solution overall: {overall_best_length:.2f}")
        print(f"Mean outer iteration time: {avg_time:.4f} sec")
        return overall_best_tour, overall_best_length


In [ ]:
import numpy as np
import random
import time
import math
from collections import defaultdict
import networkx as nx
from scipy.sparse import csr_matrix
from scipy.sparse.csgraph import minimum_spanning_tree

class TSP_LKH_Hybrid_naive:
    def __init__(self, distance_matrix):
        """
        Initialize the solver with a distance matrix.

        Args:
            distance_matrix: 2D numpy array where distance_matrix[i][j] is the distance from city i to city j
        """
        self.distances = distance_matrix
        self.num_cities = len(distance_matrix)
        self.best_tour = None
        self.best_length = float('inf')

    def calculate_tour_length(self, tour):
        """Calculate the total length of a tour."""
        length = 0
        for i in range(len(tour)):
            length += self.distances[tour[i]][tour[(i + 1) % len(tour)]]
        return length

    def nearest_neighbor_tour(self, start_city=None):
        """Generate an initial tour using the nearest neighbor heuristic."""
        if start_city is None:
            start_city = random.randint(0, self.num_cities - 1)

        unvisited = set(range(self.num_cities))
        tour = [start_city]
        unvisited.remove(start_city)

        while unvisited:
            last = tour[-1]
            # Find the nearest unvisited city
            nearest = min(unvisited, key=lambda city: self.distances[last][city])
            tour.append(nearest)
            unvisited.remove(nearest)

        return tour

    def two_opt_move(self, tour, i, j):
        """Perform a 2-opt move by reversing the segment between positions i and j."""
        if i > j:
            i, j = j, i
        new_tour = tour.copy()
        new_tour[i:j+1] = reversed(tour[i:j+1])
        return new_tour

    def lin_kernighan_step(self, tour, max_depth=5):
        """
        Perform one step of the Lin-Kernighan algorithm.

        Args:
            tour: Current tour
            max_depth: Maximum search depth

        Returns:
            Improved tour if found, otherwise None
        """
        n = len(tour)
        best_improvement = 0
        best_move = None

        # For each city
        for i in range(n):
            t1 = i
            t2 = (i + 1) % n
            city1 = tour[t1]
            city2 = tour[t2]

            # Break the link between city1 and city2
            removed_distance = self.distances[city1][city2]

            # Try to find alternate connections
            for j in range(n):
                if j == t1 or j == t2 or j == (t1 - 1) % n:
                    continue

                city3 = tour[j]
                city4 = tour[(j + 1) % n]

                # Consider joining city1-city3 and city2-city4
                new_distance = self.distances[city1][city3] + self.distances[city2][city4]

                # If this would be an improvement
                improvement = removed_distance - new_distance
                if improvement > best_improvement:
                    # This would create a valid tour by reconnecting
                    new_tour = self.reconnect_tour(tour, t1, t2, j, (j + 1) % n)
                    if new_tour is not None:
                        best_improvement = improvement
                        best_move = new_tour

        # If no improvement was found
        if best_improvement <= 0:
            return None

        return best_move

    def reconnect_tour(self, tour, t1, t2, t3, t4):
        """
        Reconnect a tour after breaking two edges.

        Args:
            tour: Current tour
            t1, t2, t3, t4: Positions of the endpoints of the edges to reconnect

        Returns:
            New tour if valid, otherwise None
        """
        # Create a simple 2-opt move
        if (t1 + 1) % len(tour) == t2 and (t3 + 1) % len(tour) == t4:
            return self.two_opt_move(tour, t2, t3)

        # More complex case - not implemented for simplicity
        # In a full LK implementation, this would handle more complex reconnections
        return None

    def lin_kernighan(self, initial_tour, max_iterations=1000):
        """
        Implement the Lin-Kernighan algorithm for TSP.

        Args:
            initial_tour: Starting tour
            max_iterations: Maximum number of iterations

        Returns:
            best_tour, best_length: The best tour found and its length
        """
        current_tour = initial_tour.copy()
        current_length = self.calculate_tour_length(current_tour)

        best_tour = current_tour.copy()
        best_length = current_length

        iterations = 0
        while iterations < max_iterations:
            # Perform one step of Lin-Kernighan
            improved_tour = self.lin_kernighan_step(current_tour)

            if improved_tour is None:
                # No improvement found
                break

            improved_length = self.calculate_tour_length(improved_tour)

            if improved_length < current_length:
                current_tour = improved_tour
                current_length = improved_length

                if current_length < best_length:
                    best_tour = current_tour.copy()
                    best_length = current_length
            else:
                # This shouldn't happen with a proper implementation, but as a safeguard
                break

            iterations += 1

        return best_tour, best_length

    def partition_graph(self, num_partitions=2):
        """
        Partition the cities into clusters using spectral clustering.

        Args:
            num_partitions: Number of clusters to create

        Returns:
            List of lists, where each inner list contains the indices of cities in one partition
        """
        # Create a graph
        G = nx.Graph()

        # Add nodes and edges
        for i in range(self.num_cities):
            G.add_node(i)

        for i in range(self.num_cities):
            for j in range(i+1, self.num_cities):
                # Use inverse of distance as weight (closer = stronger connection)
                weight = 1.0 / (self.distances[i][j] + 1e-10)
                G.add_edge(i, j, weight=weight)

        # Perform spectral clustering
        try:
            # Get the Laplacian matrix
            laplacian = nx.normalized_laplacian_matrix(G)
            eigenvalues, eigenvectors = np.linalg.eigh(laplacian.toarray())

            # Use the eigenvector corresponding to the second smallest eigenvalue for bisection
            indices = eigenvalues.argsort()

            # For k-way partitioning, use k eigenvectors
            k = min(num_partitions, self.num_cities - 1)
            fiedler_vectors = eigenvectors[:, indices[1:k+1]]

            # Use k-means clustering on the eigenvectors
            from sklearn.cluster import KMeans
            kmeans = KMeans(n_clusters=num_partitions, random_state=0).fit(fiedler_vectors)

            # Get the partitions
            partitions = [[] for _ in range(num_partitions)]
            for i, label in enumerate(kmeans.labels_):
                partitions[label].append(i)

            return partitions
        except Exception as e:
            # Fallback to simple partitioning if spectral clustering fails
            print(f"Spectral clustering failed: {e}. Using fallback partitioning.")
            return self.fallback_partition(num_partitions)

    def fallback_partition(self, num_partitions):
        """Simple geographic partitioning as a fallback."""
        # Create a simple geographic partitioning
        cities_per_partition = self.num_cities // num_partitions
        partitions = [[] for _ in range(num_partitions)]

        for i in range(self.num_cities):
            partition_idx = min(i // cities_per_partition, num_partitions - 1)
            partitions[partition_idx].append(i)

        return partitions

    def solve_partition(self, cities):
        """
        Solve TSP for a subset of cities.

        Args:
            cities: List of city indices

        Returns:
            best_tour, best_length: The best tour found and its length
        """
        if len(cities) <= 3:
            # For very small partitions, just return the cities in order
            return cities, self.calculate_tour_length(cities)

        # Create a distance matrix for the subset
        n = len(cities)
        sub_distances = np.zeros((n, n))

        for i in range(n):
            for j in range(n):
                sub_distances[i][j] = self.distances[cities[i]][cities[j]]

        # Create a temporary solver for this subset
        sub_solver = TSP_LKH_Hybrid_naive(sub_distances)

        # Generate an initial tour using nearest neighbor
        initial_tour = sub_solver.nearest_neighbor_tour()

        # Apply Lin-Kernighan to improve the tour
        sub_tour, sub_length = sub_solver.lin_kernighan(initial_tour)

        # Map the sub-tour back to the original city indices
        original_tour = [cities[idx] for idx in sub_tour]

        return original_tour, sub_length

    def merge_tours(self, tours):
        """
        Merge multiple sub-tours into a single tour.

        Args:
            tours: List of tours to merge

        Returns:
            Merged tour
        """
        if len(tours) == 1:
            return tours[0]

        # Start with the first tour
        merged_tour = tours[0].copy()
        merged_length = self.calculate_tour_length(merged_tour)

        # Merge each subsequent tour
        for next_tour in tours[1:]:
            best_merged = None
            best_merged_length = float('inf')

            # Try inserting the next tour at each possible position
            for i in range(len(merged_tour)):
                # Try inserting the next tour between positions i and i+1
                for j in range(len(next_tour)):
                    # Try starting the next tour at position j
                    candidate = merged_tour[:i+1] + next_tour[j:] + next_tour[:j] + merged_tour[i+1:]
                    candidate_length = self.calculate_tour_length(candidate)

                    if candidate_length < best_merged_length:
                        best_merged = candidate
                        best_merged_length = candidate_length

            if best_merged:
                merged_tour = best_merged
                merged_length = best_merged_length

        return merged_tour

    def improve_merged_tour(self, tour, max_iterations=1000):
        """
        Apply local search to improve a merged tour.

        Args:
            tour: Tour to improve
            max_iterations: Maximum number of iterations

        Returns:
            Improved tour and its length
        """
        # Implement 2-opt local search
        best_tour = tour.copy()
        best_length = self.calculate_tour_length(best_tour)

        improvement = True
        iterations = 0

        while improvement and iterations < max_iterations:
            improvement = False
            iterations += 1

            for i in range(len(tour) - 2):
                for j in range(i + 2, len(tour)):
                    if i == 0 and j == len(tour) - 1:
                        continue

                    new_tour = self.two_opt_move(best_tour, i, j)
                    new_length = self.calculate_tour_length(new_tour)

                    if new_length < best_length:
                        best_tour = new_tour
                        best_length = new_length
                        improvement = True
                        break

                if improvement:
                    break

        # Apply Lin-Kernighan for further improvement
        best_tour, best_length = self.lin_kernighan(best_tour, max_iterations=max_iterations//10)

        return best_tour, best_length

    def solve(self, num_partitions=4, max_iterations=1000):
        """
        Solve TSP using the hybrid approach with partitioning.

        Args:
            num_partitions: Number of partitions to create
            max_iterations: Maximum number of iterations for local search

        Returns:
            best_tour, best_length: The best tour found and its length
        """
        start_time = time.time()

        # Adjust number of partitions based on problem size
        actual_partitions = min(num_partitions, max(2, self.num_cities // 50))

        print(f"Partitioning the graph into {actual_partitions} clusters...")
        partitions = self.partition_graph(num_partitions=actual_partitions)

        # Solve each partition
        sub_tours = []
        print("Solving partitions:")
        for i, partition in enumerate(partitions):
            print(f"  Partition {i+1}/{actual_partitions} with {len(partition)} cities...")
            sub_tour, sub_length = self.solve_partition(partition)
            sub_tours.append(sub_tour)

        # Merge the sub-tours
        print("Merging sub-tours...")
        merged_tour = self.merge_tours(sub_tours)

        # Apply final improvement
        print("Applying final improvement...")
        best_tour, best_length = self.improve_merged_tour(merged_tour, max_iterations)

        solve_time = time.time() - start_time
        print(f"Solved in {solve_time:.2f} seconds")
        print(f"Best tour length: {best_length:.2f}")

        self.best_tour = best_tour
        self.best_length = best_length

        return best_tour, best_length


# Example usage
# if __name__ == "__main__":
#     # Create a random problem instance
#     num_cities = 200
#     np.random.seed(42)  # For reproducibility

#     # Generate random city coordinates
#     cities = np.random.rand(num_cities, 2) * 1000

#     # Calculate Euclidean distances
#     distance_matrix = np.zeros((num_cities, num_cities))
#     for i in range(num_cities):
#         for j in range(num_cities):
#             if i != j:
#                 distance_matrix[i][j] = np.sqrt(np.sum((cities[i] - cities[j])**2))

#     # Solve using the hybrid approach
#     solver = TSP_LKH_Hybrid(distance_matrix)

#     # Determine number of partitions based on problem size
#     num_partitions = max(2, num_cities // 50)

#     # Run the hybrid solver
#     best_tour, best_length = solver.solve(num_partitions=num_partitions)

#     print(f"Best tour length: {best_length:.2f}")

## Бенчмарк на датасете TSPLIB

In [ ]:
# Create the benchmark environment
benchmark = TSPBenchmark()

# Download some standard instances
benchmark.download_tsplib_set()

# Define solvers to benchmark
solvers = [
    ("Branch&Cut (pure)", TSP_Branch_Cut, {}),
    ("LK + Partitioning", TSP_LKH_Hybrid_naive, {
        "num_partitions": 4,
        "max_iterations": 1000
    }),
    ("2-opt + SA", TSP_2opt_SA_naive, {
        "sa_params": {
            "initial_temp": 100.0,
            "cooling_rate": 0.995,
            "max_iterations": 10000,
            "min_temp": 1e-6,  # Add min_temp to sa_params
            'max_iterations_without_improvement': 500 # Add max_iterations_without_improvement
        },
        "local_search_after_sa": True
    }),
    ("2-opt + SA (outer)", TSP_2opt_SA_outer, {
        "sa_params": {
            'initial_temp': 500.0,
            'cooling_rate': 0.9995,
            'min_temp': 1e-6,
            'max_iterations': 10000,
            'max_no_improve': 1000,
            'perturb_interval': 500,
            'perturb_prob': 0.1
        },
        "outer_iterations": 10,
        "local_search_after_sa": True,
        "use_perturbation": True
    }),
    ("2-opt + 3-opt + SA", TSP_2opt_3opt_SA, {
        "sa_params": {
            'initial_temp': 500.0,
            'cooling_rate': 0.9995,
            'min_temp': 1e-6,
            'max_iterations': 10000,
            'max_no_improve': 1000,
            'perturb_interval': 500,
            'perturb_prob': 0.1
        },
        "outer_iterations": 10,
        "local_search_after_sa": True,
        "use_3opt": True,
        "use_perturbation": True
    }),
]
instances = ["berlin52", "eil101", "ch130", "ch150", "a280", "pcb442"]

time_limits_per_instance = {
        "berlin52": 30,      # 30 секунд
        "eil101": 120,     # 2 минуты
        "ch130": 300,      # 5 минут
        "ch150": 350,      # 5 минут
        "a280": 700,     # 10 минут
        "pcb442": 3000,    # 20 минут
      }

# Benchmark on a few standard instances
benchmark_results = benchmark.benchmark_all(
    solvers=solvers,
    instances=instances,
    runs=3,  # Количество запусков для усреднения (уменьшено для примера)
    instance_time_limits=time_limits_per_instance # Передаем лимиты
)

# 6. (Опционально) Дополнительный анализ или сохранение результатов
print("\nRaw Results:", benchmark_results)

import json
# Конвертируем туры numpy/list в списки для JSON
def make_json_serializable(obj):
    if isinstance(obj, np.ndarray): return obj.tolist()
    if isinstance(obj, list): return [make_json_serializable(item) for item in obj]
    if isinstance(obj, dict): return {k: make_json_serializable(v) for k, v in obj.items()}
    if isinstance(obj, tuple): return tuple(make_json_serializable(item) for item in obj)
    # Добавьте другие типы по необходимости (Path и т.д.)
    if isinstance(obj, Path): return str(obj)
    if isinstance(obj, (int, float, str, bool)) or obj is None: return obj
    return repr(obj) # Запасной вариант

# Преобразуем ключи-кортежи в строки
string_key_results = {str(k): make_json_serializable(v) for k, v in benchmark_results.items()}

BASE_PATH = './benchmark_data/'
import uuid
hash = uuid.uuid4().hex

try:
    with open(f"{BASE_PATH}tsplib_outputs/benchmark_results_{hash}.json", "w") as f:
        json.dump(string_key_results, f, indent=4)
    print("\nBenchmark results saved to benchmark_results.json")
except Exception as e:
    print(f"\nError saving results to JSON: {e}")


# Visualize the best tour for berlin52
for instance in instances:
  for solver in solvers:
    best_result = benchmark_results[(instance, solver[0])]
    benchmark.visualize_tour(
        instance,
        best_result["best_tour"],
        f"Best tour for {instance} (length: {best_result['best_length']:.2f})"
    )

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import ast
import io
import json

# ---- 1. Загрузка данных ----
with open(f'{BASE_PATH}tsplib_outputs/benchmark_results_{hash}.json', 'r') as f:
    data = json.load(f)

# ---- 2. Парсинг данных ----
parsed_data = []
for key_str, result in data.items():
    try:
        # Безопасно парсим ключ ('instance', 'solver')
        instance_name, solver_name = ast.literal_eval(key_str)

        # Проверяем, совпадает ли имя солвера в ключе и в данных (на всякий случай)
        if solver_name != result.get("solver"):
             print(f"Предупреждение: Имя солвера в ключе '{solver_name}' "
                   f"не совпадает с именем в данных '{result.get('solver')}' "
                   f"для инстанса '{instance_name}'. Используется имя из ключа.")

        # Извлекаем нужные метрики
        parsed_entry = {
            "Инстанс": instance_name,
            "Солвер": solver_name,
            "Размерность": result.get("dimension"),
            "Лучшая Длина": result.get("best_length"),
            "Средняя Длина": result.get("mean_length"),
            "Лучший Gap (%)": result.get("best_gap"),
            "Средний Gap (%)": result.get("mean_gap"),
            "Среднее Время (с)": result.get("mean_time"),
            "Таймауты": result.get("timeouts", 0), # Добавим на всякий случай
            "Кол-во запусков": len(result.get("runs", [])), # Добавим кол-во запусков
            # "Параметры": result.get("params") # Можно раскомментировать, если нужны параметры
        }
        parsed_data.append(parsed_entry)

    except Exception as e:
        print(f"Ошибка парсинга записи с ключом '{key_str}': {e}")
        # Можно добавить более детальную обработку ошибок

# ---- 3. Создание DataFrame ----
df = pd.DataFrame(parsed_data)

# Сортировка для лучшего восприятия (сначала по инстансу, потом по солверу)
df = df.sort_values(by=["Инстанс", "Солвер"])

# Установка опций форматирования для чисел с плавающей точкой
pd.options.display.float_format = '{:.2f}'.format

# ---- 4. Вывод таблицы ----
print("--- Сводная таблица результатов бенчмарка ---")
# Используем to_string(), чтобы вывести все строки/колонки, если их много
print(df.to_string(index=False))
print("-" * 50) # Разделитель

# ---- 5. Визуализация ----

# Убедимся, что данные не пустые перед построением графиков
if not df.empty:
    # Настройка стиля графиков
    sns.set_theme(style="whitegrid")

    # --- График 1: Сравнение лучшего Gap (%) ---
    plt.figure(figsize=(12, 7)) # Размер графика
    barplot_gap = sns.barplot(
        data=df,
        x="Инстанс",
        y="Лучший Gap (%)",
        hue="Солвер", # Разделение по солверам (разные цвета)
        palette="viridis" # Цветовая палитра
    )
    plt.title('Сравнение качества решений (Лучший Gap %)')
    plt.ylabel('Лучший Gap (%) - меньше = лучше')
    plt.xlabel('Инстанс TSP')
    plt.xticks(rotation=45, ha='right') # Поворот подписей оси X для читаемости
    # Добавление значений над столбцами
    for container in barplot_gap.containers:
        barplot_gap.bar_label(container, fmt='%.2f')
    plt.tight_layout() # Автоматическая подгонка элементов графика
    plt.show()

    # --- График 2: Сравнение среднего времени выполнения ---
    plt.figure(figsize=(12, 7))
    barplot_time = sns.barplot(
        data=df,
        x="Инстанс",
        y="Среднее Время (с)",
        hue="Солвер",
        palette="magma"
    )
    plt.title('Сравнение производительности (Среднее Время)')
    plt.ylabel('Среднее Время (с) - меньше = быстрее')
    plt.xlabel('Инстанс TSP')
    plt.xticks(rotation=45, ha='right')
    # Добавление значений над столбцами
    for container in barplot_time.containers:
        barplot_time.bar_label(container, fmt='%.2f')
    plt.tight_layout()
    plt.show()

else:
    print("Нет данных для визуализации.")

In [ ]:
import json
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from typing import List, Optional, Tuple, Dict, Any
import ast # Для безопасного парсинга ключа ('instance', 'solver')

# ----- НАСТРОЙКИ -----
BENCHMARK_DIR = "./benchmark_data/tsplib_outputs" # Папка с файлами JSON
# Укажите ИМЕНА ДВУХ файлов для сравнения (без расширения .json)

FILE_1_NAME = "benchmark_results"
FILE_2_NAME = "benchmark_results_557157947aa74a258b1a59c1206f9459"

# Имена для легенды на графиках
LABEL_1 = "Original"   # Метка для данных из FILE_1
LABEL_2 = "Optimized"  # Метка для данных из FILE_2

# Порог Gap для визуализации (для ограничения оси Y)
GAP_YLIM_MAX = 25 # Максимальный Gap в %, который будет виден на графике
# ---------------------


def load_benchmark_data(filename: str) -> Optional[Dict[Tuple[str, str], Dict[str, Any]]]:
    """Загружает и парсит данные бенчмарка из JSON файла."""
    filepath = os.path.join(BENCHMARK_DIR, f"{filename}.json")
    if not os.path.exists(filepath):
        print(f"Ошибка: Файл не найден - {filepath}")
        return None
    try:
        with open(filepath, 'r') as f:
            data_str_keys = json.load(f)
        # Преобразуем строковые ключи обратно в кортежи
        data_tuple_keys = {ast.literal_eval(k): v for k, v in data_str_keys.items()}
        print(f"Данные успешно загружены из {filepath}")
        return data_tuple_keys
    except Exception as e:
        print(f"Ошибка загрузки или парсинга файла {filepath}: {e}")
        return None

def results_to_dataframe(results_dict: Dict[Tuple[str, str], Dict[str, Any]], label: str) -> pd.DataFrame:
    """Преобразует словарь результатов в Pandas DataFrame."""
    data_list = []
    for (instance, solver), result_data in results_dict.items():
        dimension = result_data.get('dimension', np.nan)
        optimal_length = result_data.get('optimal_length', np.nan) # Optimal length хранится в result_data
        best_len = result_data.get('best_length', float('inf'))
        mean_time = result_data.get('mean_time', float('inf'))

        gap = np.nan
        if optimal_length is not None and not np.isnan(optimal_length) and \
           best_len != float('inf') and optimal_length > 0:
            gap = (best_len - optimal_length) / optimal_length * 100

        data_list.append({
            "Instance": instance,
            "Dimension": dimension,
            "Solver": solver,
            "Label": label, # Добавляем метку для различения данных
            "Best Length": best_len if best_len != float('inf') else np.nan,
            "Mean Time (s)": mean_time if mean_time != float('inf') else np.nan,
            "Gap (%)": gap,
        })
    df = pd.DataFrame(data_list)
    df.dropna(subset=['Dimension'], inplace=True)
    df['Dimension'] = df['Dimension'].astype(int)
    return df

# --- Загрузка данных ---
print("Загрузка данных для сравнения...")
results1 = load_benchmark_data(FILE_1_NAME)
results2 = load_benchmark_data(FILE_2_NAME)

# --- Создание DataFrame ---
if results1 and results2:
    df1 = results_to_dataframe(results1, LABEL_1)
    df2 = results_to_dataframe(results2, LABEL_2)

    # Объединяем два DataFrame для удобства построения графиков
    combined_df = pd.concat([df1, df2], ignore_index=True)

    # Убираем солверы, которые не дали валидных результатов ни в одном из запусков
    valid_solvers = combined_df.dropna(subset=['Best Length'])['Solver'].unique()
    combined_df = combined_df[combined_df['Solver'].isin(valid_solvers)]

    # Сортируем для консистентности
    combined_df.sort_values(by=["Instance", "Solver", "Label"], inplace=True)

    print("\n--- Combined DataFrame for Plotting ---")
    print(combined_df.to_string())

    # --- Визуализация Сравнения ---
    if not combined_df.empty:
        print("\nGenerating Comparison Plots...")
        sns.set_theme(style="whitegrid")

        # Получаем список уникальных инстансов и солверов
        instances = sorted(combined_df['Instance'].unique(), key=lambda x: combined_df[combined_df['Instance']==x]['Dimension'].iloc[0])
        solvers = sorted(combined_df['Solver'].unique())

        # Цвета для двух сравниваемых запусков
        palette = {"Original": "skyblue", "Optimized": "salmon"} # Настройте цвета
        # Или используйте стандартную палитру: palette = sns.color_palette("viridis", 2)

        # --- График 1: Сравнение Gap (%) с наложением ---
        plt.figure(figsize=(max(10, len(instances)*2), 7)) # Ширина зависит от числа инстансов
        barplot_gap = sns.barplot(
            data=combined_df,
            x="Instance",
            y="Gap (%)",
            hue="Label", # Разделяем столбцы по метке (Original/Optimized)
            palette=palette, # Используем заданные цвета
            order=instances # Задаем порядок инстансов по размерности
        )
        plt.title('Сравнение Качества Решений (Лучший Gap %)')
        plt.ylabel('Лучший Gap (%) - меньше = лучше')
        plt.xlabel('Инстанс TSP')
        plt.xticks(rotation=45, ha='right')
        plt.ylim(bottom=min(0, combined_df['Gap (%)'].min(skipna=True) - 1), top=GAP_YLIM_MAX) # Ограничиваем ось Y
        plt.grid(axis='y', linestyle='--', alpha=0.7)
        # Добавляем значения (опционально, может быть тесно)
        # for container in barplot_gap.containers:
        #     barplot_gap.bar_label(container, fmt='%.2f', fontsize=8)
        plt.legend(title="Запуск")
        plt.tight_layout()
        plt.show()

        # --- График 2: Сравнение Времени (с наложением) ---
        plt.figure(figsize=(max(10, len(instances)*2), 7))
        barplot_time = sns.barplot(
            data=combined_df,
            x="Instance",
            y="Mean Time (s)",
            hue="Label",
            palette=palette,
            order=instances
        )
        plt.title('Сравнение Производительности (Среднее Время)')
        plt.ylabel('Среднее Время (с) - меньше = быстрее')
        plt.xlabel('Инстанс TSP')
        plt.xticks(rotation=45, ha='right')
        plt.yscale('log') # Логарифмическая шкала для времени
        plt.grid(axis='y', linestyle='--', alpha=0.7)
        # Добавляем значения (опционально)
        # for container in barplot_time.containers:
        #    barplot_time.bar_label(container, fmt='%.2f', fontsize=8)
        plt.legend(title="Запуск")
        plt.tight_layout()
        plt.show()

        # --- График 3: Scatter Plot Время vs Качество (точки для обоих запусков) ---
        plt.figure(figsize=(12, 8))
        scatter_plot = sns.scatterplot(
            data=combined_df.dropna(subset=['Mean Time (s)', 'Gap (%)']), # Убираем NaN
            x='Mean Time (s)',
            y='Gap (%)',
            hue='Solver',   # Цвет по солверу
            style='Label',  # Форма маркера по типу запуска (Original/Optimized)
            s=150,          # Размер маркеров
            palette='tab10', # Палитра для солверов
            legend='full'
        )
        plt.title('Компромисс Время-Качество (Сравнение Запусков)', fontsize=16)
        plt.xlabel('Среднее Время Решения (s)')
        plt.ylabel('Среднее Отклонение от Оптимума (%)')
        plt.grid(True, linestyle='--', alpha=0.7)
        plt.axhline(0, color='grey', linestyle=':', linewidth=1)
        # plt.xscale('log') # Опционально лог. шкала времени
        # Перемещаем легенду
        plt.legend(bbox_to_anchor=(1.02, 1), loc='upper left', borderaxespad=0.)
        plt.tight_layout(rect=[0, 0, 0.85, 1]) # Оставляем место для легенды
        plt.show()

    else:
        print("DataFrame пуст, графики не будут построены.")
else:
    print("Не удалось загрузить один или оба файла JSON для сравнения.")